# Project 3 - Search-Based Web Fuzzer 

In [ ]:
import bookutils.setup

## Introduction

Fuzzing web applications can be challenging. A simple web form requires a specific input format for each field, this format needs to be fulfilled for a valid web interaction (e.g. an email field is expected to fulfill the following regex pattern `r'^([a-z0-9_\.-]+)@([\da-z\.-]+)\.([a-z\.]{2,6})$')`.

In this project, we demonstrate how to apply a search-based algorithm to generate specific test inputs for web interfaces. The generated inputs should exercise the web application by fulfilling the input validation requirements of all fields required to reach specific web pages (both normal and error pages). 

The task is to employ a genetic algorithm (GA) for fuzzing, your implementation is expected to generate inputs that match the expected pattern or constraints of the input field (e.g. email fuzzingbook@gmail.com fulfils the expected email regex pattern `r'^([a-z0-9_\.-]+)@([\da-z\.-]+)\.([a-z\.]{2,6})$')`. This is to be achieved by searching the input space starting from an initially (random) input. 

In [ ]:
from WebFuzzer import init_db, ORDERS_DB, SimpleHTTPRequestHandler  
from http.server import HTTPServer, HTTPStatus 

### Objective

The goal of this project is to _implement a search-based fuzzing algorithm that generates specifically formatted inputs which fulfil the input validation requirements of a web form_. 

You will apply techniques learned in the lecture ([SBST](SBST.ipynb) and [WebFuzzer](WebFuzzer.ipynb)) to automatically fill web forms by producing inputs that improve the reachability of web pages. 

The goal of your Fuzzer is to cleverly search the input space until the input validation format of the input field is achieved. Consequently, fulfilling (or not fulfilling) such input validation schemes achieves the reachability of specific web pages and improves the coverage of the site map. 

# Web Application

We create a (HTML only) web application for placing an order of items, similar to the order form in the [WebFuzzer](WebFuzzer.ipynb) lecture.


## Input Validation

We create a `ProjectHTTPRequestHandler` class that provides a web order form with input validation schemes for all input formats. 

First, we create a sample set of regex for input validation of each input field in our order form. 

__Note that your implementation shall be evaluated with a _secret set of regex_ other than the ones provided below.__

For some input fields, we have provided other sample regexes you may use to test your implementation, other regexes are provided to inform you of the nature of the regexes which may be used to evaluate your final solution.  


In [ ]:
itemList = r'^(tshirt|drill|lockset)$'
name_regex = [r'^[A-Z][a-z]+ [A-Z][a-z]+ [A-Z][a-z]+$', r'^[A-Z][a-z]+ [A-Z]\. [A-Z][a-z]+$', \
              r'^[A-Z][a-z]+ [A-Z][a-z]+$', r'[a-z]+ [a-z]+$']
email_regex = [r'^([a-z0-9_\.-]+)@([\da-z\.-]+)\.([a-z\.]{2,6})$', r'^([a-z0-9_\.-]+)@cispa\.saarland$', \
              r'^([a-z0-9_\.-]+)@gmail\.com$', r'^([a-z0-9_\.-]+)@hotmail\.co\.uk$']
zip_regex = [r'^\d{5}$', r'\d{3}-\d{4}',  r'\d{5}([ \-]\d{4})?', \
             r'[ABCEGHJKLMNPRSTVXY]\d[ABCEGHJ-NPRSTV-Z][ ]?\d[ABCEGHJ-NPRSTV-Z]\d']
city_regex = [r'[A-Z]\w{2,9}$', r'^[a-zA-Z]+(?:[\s-][a-zA-Z]+)*$', r'^\p{Lu}\p{L}*(?:[\s-]\p{Lu}\p{L}*)*$', r'\w{2,12}$']
terms_regex = r'on'

list_regex = [itemList, name_regex[0], email_regex[0], city_regex[0], zip_regex[0], terms_regex]

We define diagnostic error messages for each error page.

In [ ]:
error_msg_list = ["INCOMPLETE INPUT: \n  All fields have to be filled ",\
                   "INVALID INPUT: \n  Item is not in item list",\
                   "INVALID INPUT: \n  Name does not match expected pattern :\n ",\
                  "INVALID INPUT: \n  Email does not match expected pattern :\n ",\
                  "INVALID INPUT: \n  City does not match expected pattern :\n ",\
                  "INVALID INPUT: \n  Zip does not match expected pattern :\n ",\
                  "INVALID INPUT: \n  Terms and Conditions have to be checked "
                  ]

We override the `handle_order()` method, to validate the values of each input field using the pre-defined regex patterns above. 

In [ ]:
import re

class ProjectHTTPRequestHandler(SimpleHTTPRequestHandler):
    def handle_order(self): 
        values = self.get_field_values()

        if len(values) < 6:
            error_type = error_msg_list[0]
            self.internal_server_error(error_type)
            
        elif re.match(list_regex[0], values["item"]) and re.match(list_regex[1], values["name"]) and \
        re.match(list_regex[2], values["email"]) and re.match(list_regex[4], values["zip"]) and \
        re.match(list_regex[3], values["city"]) and re.match(list_regex[5], values["terms"]):
            self.store_order(values)
            self.send_order_received(values)

        elif not re.match(list_regex[0], values["item"]):
            error_type = error_msg_list[1]
            self.internal_server_error(error_type) 
        
        elif not re.match(list_regex[1], values["name"]):
            error_type = error_msg_list[2] + name_regex[0] 
            self.internal_server_error(error_type)
        
        elif not re.match(list_regex[2], values["email"]):
            error_type = error_msg_list[3] + email_regex[0]
            self.internal_server_error(error_type)
            
        elif not re.match(list_regex[3], values["city"]):
            error_type = error_msg_list[5] + city_regex[0]
            self.internal_server_error(error_type)
        
        elif not re.match(list_regex[4], values["zip"]):
            error_type = error_msg_list[4] + zip_regex[0]
            self.internal_server_error(error_type)
    
        elif not re.match(list_regex[5], values["terms"]):
            error_type = error_msg_list[6]
            self.internal_server_error(error_type)
        

### Internal Errors
For diagnostic (and identification) purposes, the internal server error pages include the error message indicating the input validation scheme that failed.

In [ ]:
HTML_INTERNAL_SERVER_ERROR = """
<html><body>
<div style="border:3px; border-style:solid; border-color:#FF0000; padding: 1em;">
  <strong id="title" style="font-size: x-large">Internal Server Error</strong>
  <p>
  The server has encountered an internal error.  Go to our <a href="/">order form</a>.
  <pre>{error_message}</pre>
  </p>
</div>
</body></html>
  """

In [ ]:
import sys
import traceback

class ProjectHTTPRequestHandler(ProjectHTTPRequestHandler):
    def internal_server_error(self, error_type=None):
        self.send_response(HTTPStatus.OK, "Internal Error")

        self.send_header("Content-type", "text/html")
        self.end_headers()

        exc = traceback.format_exc()
        self.log_message("%s", exc.strip())

        message = HTML_INTERNAL_SERVER_ERROR.format(error_message=error_type)
        self.wfile.write(message.encode("utf8"))

In [ ]:
from bookutils import HTML 
HTML(HTML_INTERNAL_SERVER_ERROR)

### Storing Orders

Similar to the [WebFuzzer](WebFuzzer.ipynb) lecture, we store orders using a *database* stored in the file `orders.db`.

In [ ]:
FUZZINGBOOK_SWAG = {
    "tshirt": "One FuzzingBook T-Shirt",
    "drill": "One FuzzingBook Rotary Hammer",
    "lockset": "One FuzzingBook Lock Set"
}

In [ ]:
import sqlite3, os

class ProjectHTTPRequestHandler(ProjectHTTPRequestHandler):
    def store_order(self, values):
        db = sqlite3.connect(ORDERS_DB)
        db.execute("INSERT INTO orders VALUES (?, ?, ?, ?, ?, ?)",
                (values['item'], values['name'], values['email'], values['city'], values['zip'], values['terms']))
        db.commit()

    def send_order_received(self, values):
        # Should use html.escape()
        values["item_name"] = FUZZINGBOOK_SWAG[values["item"]]
        confirmation = HTML_ORDER_RECEIVED.format(**values).encode("utf8")

        self.send_response(HTTPStatus.OK, "Order received")
        self.send_header("Content-type", "text/html")
        self.end_headers()
        self.wfile.write(confirmation)
        
"""

    def send_order_received(self, values):
        sanitized_values = {}
        for field in values:
            sanitized_values[field] = html.escape(values[field])
        sanitized_values["item_name"] = html.escape(
            FUZZINGBOOK_SWAG[values["item"]])

        confirmation = HTML_ORDER_RECEIVED.format(
            **sanitized_values).encode("utf8")

        self.send_response(HTTPStatus.OK, "Order received")
        self.send_header("Content-type", "text/html")
        self.end_headers()
        self.wfile.write(confirmation)
"""

In [ ]:
def init_db():
    if os.path.exists(ORDERS_DB):
        os.remove(ORDERS_DB)

    db_connection = sqlite3.connect(ORDERS_DB)
    db_connection.execute("DROP TABLE IF EXISTS orders")
    db_connection.execute("CREATE TABLE orders (item text, name text, email text, city text, zip text, terms text)")
    db_connection.commit()

    return db_connection

db = init_db()

In [ ]:
print(db.execute("SELECT * FROM orders").fetchall())

db.execute("INSERT INTO orders " +
           "VALUES ('lockset', 'Jane Lee Doe', 'jane@doe.com', 'Saarbrucken', '66125', 'on')")
db.commit()

In [ ]:
print(db.execute("SELECT * FROM orders").fetchall())


db.execute("DELETE FROM orders WHERE name = 'Walter White'")
db.commit()

In [ ]:
print(db.execute("SELECT * FROM orders").fetchall())

### Order Confirmation

Once we have gotten an order, we show a confirmation page, which is instantiated with the customer information submitted before.

In [ ]:
HTML_ORDER_RECEIVED = """
<html><body>
<div style="border:3px; border-style:solid; border-color:#FF0000; padding: 1em;">
  <strong id="title" style="font-size: x-large">Thank you for your Fuzzingbook Order!</strong>
  <p id="confirmation">
  We will send <strong>{item_name}</strong> to {name} in {city}, {zip}<br>
  A confirmation mail will be sent to {email}.
  </p>
  <p>
  Want more swag?  Use our <a href="/">order form</a>!
  </p>
</div>
</body></html>
"""

### Logging 

Similar to the [WebFuzzer](WebFuzzer.ipynb) lecture, we provide a logging infrastructure for debugging purposes. We implement the following:
- `print_httpd_messages()` prints all messages accumulated in the queue
- `clear_httpd_messages()` discards all pending messages
- `log_message()` store messages in the queue 

In [ ]:
from multiprocess import Queue, Process

In [ ]:
from bookutils import rich_output, terminal_escape, unicode_escape, HTML

In [ ]:
HTTPD_MESSAGE_QUEUE = Queue()

In [ ]:
def display_httpd_message(message):
    if rich_output():
        display(
            HTML(
                '<pre style="background: NavajoWhite;">' +
                message +
                "</pre>"))
    else:
        print(terminal_escape(message))

In [ ]:
def print_httpd_messages():
    while not HTTPD_MESSAGE_QUEUE.empty():
        message = HTTPD_MESSAGE_QUEUE.get()
        display_httpd_message(message)

In [ ]:
def clear_httpd_messages():
    while not HTTPD_MESSAGE_QUEUE.empty():
        HTTPD_MESSAGE_QUEUE.get()

In [ ]:
class ProjectHTTPRequestHandler(ProjectHTTPRequestHandler):
    def log_message(self, format, *args):
        message = ("%s - - [%s] %s\n" %
                   (self.address_string(),
                    self.log_date_time_string(),
                    format % args))
        HTTPD_MESSAGE_QUEUE.put(message)

Extend `webbrowser()` method to prints log messages produced by the server:

In [ ]:
from Carver import webbrowser as simple_webbrowser

def webbrowser(url, mute=False):
    try:
        contents = simple_webbrowser(url)
    finally:
        if not mute:
            print_httpd_messages()
        else:
            clear_httpd_messages()

    return contents

## Running the Web Application
Similar to the [WebFuzzer](WebFuzzer.ipynb) lecture, to run the web application we implement the following:

* `run_httpd_forever()`
* `start_httpd()`
* `print_url()`


In [ ]:
def run_httpd_forever(handler_class):
    host = "127.0.0.1"  # localhost IP
    for port in range(8800, 9000):
        httpd_address = (host, port)

        try:
            httpd = HTTPServer(httpd_address, handler_class)
            break
        except OSError:
            continue

    httpd_url = "http://" + host + ":" + repr(port)
    HTTPD_MESSAGE_QUEUE.put(httpd_url)
    httpd.serve_forever()

In [ ]:
def start_httpd(handler_class=ProjectHTTPRequestHandler):
    clear_httpd_messages()

    httpd_process = Process(target=run_httpd_forever, args=(handler_class,))
    httpd_process.start()

    httpd_url = HTTPD_MESSAGE_QUEUE.get()
    return httpd_process, httpd_url

In [ ]:
httpd_process, httpd_url = start_httpd(ProjectHTTPRequestHandler)

In [ ]:
def print_url(url):
    if rich_output():
        display(HTML('<pre><a href="%s">%s</a></pre>' % (url, url)))
    else:
        print(terminal_escape(url))

In [ ]:
print_url(httpd_url)

**Note**: The above URL only works if you are running the Jupyter notebook server on the local host.

## Testing the Web Application
### Input Format

When the user clicks `Submit` on the order form, the Web browser creates and retrieves a URL of the form:

```
<hostname>/order?item=value_1&name=value_2&email=value_3&city=value_4&zip=value_5&terms=value_6
```

We test standard behavior:

In [ ]:
standard_order = "/order?item=tshirt&name=Jane+Lee+Doe&email=doe%40example.com&city=Seattle&zip=98104&terms=on"
contents = webbrowser(httpd_url + standard_order)
HTML(contents)

In [ ]:
assert contents.find("Thank you") and contents.find("We will send") and contents.find("A confirmation mail will be sent to") > 0

We test erroneous behavior - terms and conditions not checked:

In [ ]:
error_order = "/order?item=tshirt&name=Jane+Lee+Doe&email=doe%40example.com&city=Seattle&zip=98104&terms=off"
contents = webbrowser(httpd_url + error_order)
HTML(contents)

In [ ]:
assert contents.find("Internal Server Error") and contents.find(error_msg_list[6]) > 0

We test incomplete order behavior:

In [ ]:
incomplete_order = "/order?item=tshirt&name=Jane+Lee+Doe&email=doe%40example.com&city=Seattle&zip=98104"
contents = webbrowser(httpd_url + incomplete_order)
HTML(contents)

In [ ]:
assert contents.find("Internal Server Error") and contents.find(error_msg_list[0]) > 0

To test for another input validaion regex (e.g. the second regex pattern for each field - index 1), use:

In [ ]:
list_regex = [itemList, name_regex[1], email_regex[1], city_regex[1], zip_regex[1], terms_regex]
httpd_process, httpd_url = start_httpd(ProjectHTTPRequestHandler)
print_url(httpd_url)
new_regex_standard_order = "/order?item=tshirt&name=Jane+L.+Doe&email=doe%40cispa.saarland&city=Seattle&zip=984-1234&terms=on"
contents = webbrowser(httpd_url + new_regex_standard_order)
HTML(contents)
assert contents.find("Thank you") and contents.find("We will send") and contents.find("A confirmation mail will be sent to") > 0

Now, we reset the server to use the first set of regex for input validation (index 0):

In [ ]:
list_regex = [itemList, name_regex[0], email_regex[0], city_regex[0], zip_regex[0], terms_regex]
httpd_process, httpd_url = start_httpd(ProjectHTTPRequestHandler)
print_url(httpd_url)


Test for the first set of regex again:

In [ ]:
contents = webbrowser(httpd_url + standard_order)
assert contents.find("Thank you") and contents.find("We will send") and contents.find("A confirmation mail will be sent to") > 0

## Coverage of Web App 
In this project, the coverage of the web app is defined as _web reachability_, i.e. the number of reached pages on the site. 

We reachability is a proxy measurement of the number of validation schemes passed/failed and also a proxy measurement of brach coverage in the `handle_order()` method. 

### Obtaining Coverage

We define the `target_list` for each web page accordingly:

In [ ]:
"""
target_list:
    ISE_Name = Internal Server Error Web Page for "Name" field error, analogously for ISE_Item, ISE_City, etc..
    ISE_INC = Internal Server Error Web Page for "Incomplete Input" 
    PNF =  Page Not Found Web Page
    CONF = CONFIRMATION_PAGE
"""

target_list = ["ISE_INC", "ISE_Item", "ISE_Name", "ISE_Email", "ISE_Zip", "ISE_City","ISE_Terms", "CONF", "PNF"]

To obtain the overall coverage achieved by the GA's resulting population, we implement the following web reachability function. 

For each target, we compute:
* _validity_ score: the number of valid inputs in the generated population that reach the target, and
* _reachability_ score: we score the number of inputs that reach the each page. 

The reachability scores for the number of inputs reaching the targeted is based on the perceived difficulty to reach the page:
* 2 points for each input reaching an error page
* 5 points for each input reaching the confirmation page
* 1 point for each input reaching the page not found page

**Note**: we expect each run of your genetic algorithm to produce 10 inputs maximum.

In [ ]:
def get_web_reachability(orders, target):
    num_reached_pages = dict() 
    reachability, validity = 0, 0
    conf_msg = ["Thank you", "We will send", "A confirmation mail will be sent to"]
    page_not_found = "This page does not exist"
    
    for order in orders:
        clear_httpd_messages()
        contents = webbrowser(httpd_url + order, mute=True)
        HTML(contents)

        # obtain 2 reachability points for each input reaching an error page
        if contents.find("Internal Server Error") > 0:
            if target_list.index(target) <= 6:
                msg = error_msg_list[target_list.index(target)]
                if contents.find(msg) > 0: 
                    if not msg in num_reached_pages:
                        num_reached_pages[msg] = 2
                    else:
                        num_reached_pages[msg] += 2
                    validity += 1

        # obtain 5 reachability  points for each input reaching confirmation page
        if target ==  target_list[7] and contents.find(conf_msg[0]) > 0 and contents.find(conf_msg[1]) > 0 \
        and contents.find(conf_msg[2]) > 0:
            if not conf_msg[0] in num_reached_pages:
                num_reached_pages[conf_msg[0]] = 5
            else:
                num_reached_pages[conf_msg[0]] += 5
            validity += 1
        
        # obtain 1 reachability  point for each input reaching a non-existent page
        if target == target_list[8] and contents.find(page_not_found) > 0:
            if not page_not_found in num_reached_pages:
                num_reached_pages[page_not_found] = 1
            else:
                num_reached_pages[page_not_found] += 1
            validity += 1
            
        
    if len(num_reached_pages) > 0:
        reachability = sum(num_reached_pages.values())
    return reachability, validity

We test the `get_web_reachability` function for the `confirmation` page target with a sample list of test order URLs. 

In [ ]:
test_orders = ["/order?item=tshirt&name=Jane+Lee+Doe&email=doe%40example.com&city=Seattle&zip=98104&terms=off",\
                  "/order?item=tshirt&name=Jane+Lee+Doe&email=doe%40example.com&city=Seattle&zip=98104&terms=on",\
                  "/order", "/none",\
                  "/order?item=lockset&name=Jane+lee+Doe&email=doe%40example.com&city=Seattle&zip=98104&terms=on",\
                  "/order?item=tshirt&name=Jane+Lee+Doe&email=doe%40cispa.saarland&city=Seattle&zip=98104&terms=on",\
                  "/order?item=book&name=Jane+Lee+Doe&email=doe%40example.com&city=Seattle&zip=98104&terms=on",\
                  "/order?item=tshirt&name=Jane+Lee+Doe&email=doe%40example.com&city=Seattle&zip=98104&terms=on",\
                  "/order?item=tshirt&name=Jane+Lee+Doe&email=doe%40example.com&city=Seattle&zip=498104&terms=on",\
                  "/order?item=tshirt&name=Jane+Lee+Doe&email=doe%40example.com&city=Neunkirchen&zip=98104&terms=on",\
                 ]

Test for the reachability of confirmation page ("CONF") using the above order inputs:

In [ ]:
get_web_reachability(test_orders, "CONF")

## Search Based Fuzzing

We demonstrate how a specific input format can be generated with a simple search-based algorithm and a naive fitness function. 

We assume an input url with an invalid name format error:

In [ ]:
import random, string
invalid_name_order = "/order?item=tshirt&name=Jane+Lee+Doe1&email=doe%40example.com&city=Seattle&zip=98104&terms=on"
contents = webbrowser(httpd_url + invalid_name_order)
HTML(contents)
assert contents.find("Internal Server Error") and contents.find(error_msg_list[2]) > 0

In [ ]:
import urllib.parse
test_name = urllib.parse.parse_qs(invalid_name_order, keep_blank_values=True)["name"][0] 

We implement a naive distance function for "name" input field:

In [ ]:
def naive_calculate_name_distance(name):
    name_regex = r'[A-Z][a-z]+ [A-Z][a-z]+ [A-Z][a-z]+'
    res = 1
    if len(name.split()) == 3 and check_ascii_letters(name) and \
        check_word_size(name) and name.istitle():
        res = 0
    else:
        if len(name.split()) == 3:
            res -= 0.4
        if check_ascii_letters(name):
            res -= 0.35
        if check_word_size(name):
            res -= 0.15
        if name.istitle():
            res -= 0.10
    return res 

def check_ascii_letters(name):
    res =  True
    for char in name.replace(" ", ""):
        if not (char in string.ascii_letters):
            res = False
            break
    return res

def check_word_size(name):
    res = True
    for word in name.split():
        if len(word) < 2:
            res = False
            break
    return res

We implement a naive fitness function for the name input. 


In [ ]:
def get_naive_fitness(name):
    global distance
    distance = naive_calculate_name_distance(name)
    fitness = distance
    return fitness

We implement a simple mutation function `flip_random_character` for the name input. 

In [ ]:
def flip_random_character(s):
    pos = random.randint(0, len(s) - 1)
    new_c = chr(random.randrange(65, 130))
    return s[:pos] + new_c + s[pos + 1:]

We implement a simple search-based algorithm -`randomized_hillclimber()`, to run for a maximum of 5000 iterations :

In [ ]:
def randomized_hillclimber(name):
    fitness = get_naive_fitness(name)
    print("Initial value: %s at fitness %.4f" % (name, fitness))
    iterations = 0

    while fitness > 0 and iterations < 5000:
        iterations += 1
        
        mutated = flip_random_character(name)
        new_fitness = get_naive_fitness(mutated)
        if new_fitness <=  fitness:
            name = mutated
            fitness = new_fitness

    if iterations < 5000:
        print("Optimum at %s after %d iterations" % (name, iterations))
    else:
        print("# iterations  is: %d, stop!!!" % (iterations))
    return name, iterations

In [ ]:
for i in range(0, 5):
    new_name, no_iterations = randomized_hillclimber(test_name)   
    if no_iterations < 5000:
        new_name = new_name.replace(" ", "+")
        new_order = "/order?item=tshirt&name=" + new_name + "&email=doe%40example.com&city=Seattle&zip=98104&terms=on"
        new_order
        break
    new_name = new_name.replace(" ", "+")
    new_order = "/order?item=tshirt&name=" + new_name + "&email=doe%40example.com&city=Seattle&zip=98104&terms=on"
    new_order

In [ ]:
clear_httpd_messages()
contents = webbrowser(httpd_url + new_order)
HTML(contents)
assert contents.find("Thank you") and contents.find("We will send") and contents.find("A confirmation mail will be sent to") > 0

# Genetic Algorithm

## Genetic Algorithm template

The basic template of the genetic algorithm is based on the [SBST](SBST.ipynb) from the lecture.

This template uses your implemented fitness, selection, mutation and crossover functions for the population of input urls. 

In [ ]:
def evaluate_population(population, target, list_regex):
    fitness = [get_fitness(x, target, list_regex) for x in population]
    return list(zip(population, fitness))

In [ ]:
def terminal_repr(s):
    return terminal_escape(repr(s))

In [ ]:
import time

def genetic_algorithm(population, target, list_regex):
    # Generate and evaluate initial population
    generation = 0
    fitness = evaluate_population(population, target, list_regex)
    best = min(fitness, key=lambda item: item[1])
    best_individual = best[0]
    best_fitness = best[1]
    #print("Best fitness of initial population: %s - %.10f" %
    #    (terminal_repr(best_individual), best_fitness))

    start_time = time.time()
    time_spent = 0
        
    # Stop when optimum found, or we run out of patience, i.e. 5000 iterations or 5 hours max
    while best_fitness > 0 and generation < 5000 and (time_spent <= 18000):

        # The next generation will have the same size as the current one
        new_population = []
        while len(new_population) < len(population):
            # Selection
            offspring1 = selection(fitness, 10)
            offspring2 = selection(fitness, 10)

            # Crossover
            if random.random() < 0.7:
                (offspring1, offspring2) = crossover(offspring1, offspring2)

            # Mutation
            offspring1 = mutate(offspring1)
            offspring2 = mutate(offspring2)

            new_population.append(offspring1)
            new_population.append(offspring2)

        # Once full, the new population replaces the old one
        generation += 1
        population = new_population
        fitness = evaluate_population(population, target, list_regex)

        best = min(fitness, key=lambda item: item[1])
        best_individual = best[0]
        best_fitness = best[1]
        
        
        end_time = time.time()
        time_spent = end_time - start_time

        #print(
        #    "Best fitness at generation %d: %s - %.8f" %
        #    (generation, terminal_repr(best_individual), best_fitness))

    #print(
    #    "Best individual: %s, fitness %.10f" %
    #    (terminal_repr(best_individual), best_fitness))
    return population, generation

## Your Tasks
For each input, you are epxected to produce a set of urls to reach a targeted web page, i.e. generate inputs that will fulfill the input validation requirements necessary to reach a specific web page. Overall, all web pages will be targets, i.e. both error and normal pages. 

Your task is to implement your own custom selection, fitness and mutation functions for the genetic algorithm, in order to fulfill the input validation and web reachabilty requirements.

## Tips

* Your fitness function should target a specific web page,e.g. a specific error page like an invalid name error page. 
* For input (fragment) distance consider using a (fuzzy/approximate) regex matching distance of the input to the expected regex.
* You can develop any type of mutation, selection and crossover functions.
* Ensure that your implementation accounts for any arbitrary regex and any random initial population of inputs.
* For debugging purposes: unmute the `webbrowser()` to obtain logging information, i.e. set `webbrowser(url, mute=False)` 
* Do not implement in any other section except the section below. 
* Gracefully handle exceptions and errors resulting from your impelementation.
* __Remember the input validation regex and initial input population could be arbitrary, do not hard code for a specific regex or input__.

## Your Code

In the following section implement the following:
* __Fitness Function(s)__:  define a (set of) fitness function for an order URL to validate/invalidate each input field to reach specific pages (target). 

* __Selection Function__: define a selection function for the given generated population of URLs.

* __Crossover Function__: define a crossover function for two parent input URLs.

* __Mutate Function__: define a mutate function for our input URL.

Implement your own fitness, selection and mutation functions for the GA algorithm below:

In [ ]:
"""
WRITE YOUR CODE BELOW
"""

#e.g. get_fitness(order), ehere targte = confirmation_page 

def get_fitness(order, target, list_regex):
    # Reset any distance values from previous executions
    global distances_true, distances_false
    distances_true = {}
    distances_false = {}

    # run the webpage with this order
    try:
        contents = webbrowser(httpd_url + order)
        HTML(contents)
    except BaseException:
        pass

    fitness = 0.0
    
    """
    WRITE YOUR CODE HERE 
    """
    
    return fitness

def selection(evaluated_population, tournament_size):
    """
    WRITE YOUR CODE HERE 
    """
    return winner

def crossover(parent1, parent2):
    """
    WRITE YOUR CODE HERE 
    """
    return (offspring1, offspring2)

def mutate(chromosome):
    """
    WRITE YOUR CODE HERE 
    """
    return mutated

# Evaluation code

The code in the following section will be used to evaluate your impelementation.

## Random Population of Inputs
We randomly generated an `initial_population` of 20 set of inputs for our order web form, with (4) varying levels of difficulty (or garbage). Each populations of input URL contains 10 input URLs for orders.

__Note that the url path population which will be used for evaluation may be different from those provided as example.__

In [ ]:
easy_population_1 = ['/order?item=lockseR&name=RG}rqM`Ak_UTHDdxZtwepzCnmp^MKIF_olhKjBvZ`Ih]`[zQl+_PVkfPqWRhM]x~fbFzKbfwrMFVH^~h+v{ko]lL|CIUVAj{_P_pXpD\\SIGpG~Ws&email=qawxodegjvdzrovtd%40cmgjm&city=Wpk^kvTLu&zip=1176&terms=on',
 '/order?item=lockseR&name=xjOpvBQETS|uWZ\\mxjgRH+Jrf[|^PJzdNv{^+je\\Q{ikyikRP&email=vtjbtbqrafmcwtkakv%40kc&city=oF&zip=0023&terms=off',
 '/order?item=lockset&name=kLQEm[+nSHuhK{JH{_DWhBZPqW~eO_IhYGUOo\\IhEwYRfRQAqeGW{Z+}OtVpB&email=hrumxvplbtrlkbl%40ss&city=rWPMDudM}&zip=68433&terms=on',
 '/order?item=drills&name=`[LRcAVCbhPKh\\{~}ijb+{sc[SzRl|GXgkduPCrAkETRvOBhjX{UjI+^PKe~S^][&email=dbtmouijqdesfvsxqpydle%40rgz&city=ipF&zip=84627&terms=off',
 '/order?item=drills&name=UbRCtoM|NpErry~fuO[Rxua]}Tm~l|LxYX{~sLYb}GVxQsGku+Vo`\\LiR|]rk_b~asyaITVb~+VYDvNGMuhgxI^w&email=uahqmmquiab%40xo&city=upuM&zip=12425&terms=off',
 '/order?item=tshirts&name=Ft~Uf`|nEXPoOM`IMuQvJ+mX\\n^gWwUwf^MRPlCi^ovEjcFg+t\\p{jFdAtZuE{sV&email=bv%40bskm&city={gqO\\NtzK&zip=5376&terms=on',
 '/order?item=drills&name=u~wLUXiowfeCzm{PKyANNDs+{lzNCGfCtQ+RYKuTLxfdTRbl\\BMK|x|PTIfQusmpeDlsuo{CV&email=mkjujsfzkjiwfrfhlfesjgfot%40ugtio&city=xw_^EtapK&zip=04571&terms=off',
 '/order?item=tshirts&name=_g[MG|eEb|eXioY\\vzn{Qqf~Ou_z`^ciVoe}mZsYFRV]~gy+hKDupKRjBuPGfCyNEaheJbuNH}cyOl{O{SJQ`}+MLOuOkdNLYhajrRLExQkxYL\\olYPAY{&email=iatfnro%40em&city=aqSuXH&zip=77570&terms=undefined',
 '/order?item=tshirts&name=xOmdibaPZimFsSwkFDGmQ+JwzL_tZFn_p^NJDqkN+GzkhZ|b|xu\\SxY&email=pncvlnkcxzk%40frj&city=tE}B]JgQl&zip=2451&terms=off',
 '/order?item=tshirt&name=zljV`SX[LRgDmk{LreZRmTsIIJ|Dyla^gtQ]LdaDAIHYN+PD~JDMJRmQd{O+Dub}IiaxwcYsHCU{^epIHr{HS~bzLRpa|q{JX]W&email=jvvzkgoxptplcjztai%40usv&city=AI&zip=5248&terms=undefined']

easy_population_2 = ['/order?item=lockset&name=NvJywJLGfQ\\`yfGcJ+JhkHSGO]dO}IuzRR`kYZtwY[Mrim+u\\FhGmSNsCzHKLnpbqZzDV[[UDJHd&email=xdhjicxqfbp%40acwuu&city=sc}&zip=7173&terms=on',
 '/order?item=drill&name=_}PFXrG]+{uwnfrZCJO~CoKSFAEE+QUGuyi&email=pxblczskyiehfoigrworlrpzh%40dbj&city=yY&zip=8724&terms=undefined',
 '/order?item=tshirts&name=AOtv~znUiNso[Zpl_[UtyKihb[eHQ~h`gJcC~BjV+[pKDvDN{][qXwFPUWxZeNXsHvctQiuyHqVPZXW}`oDoidcg+VyTqltmD`iOH~~~kirCPakD&email=vfh%40jz&city=Qqd\\&zip=61250&terms=on',
 '/order?item=lockSet&name=eZjnTDD[cc|HWgcF_yDCh]OpbfOwExjYbaYrKqrHh~\\|x+X_TfD_oMV^NHoziaQoE~wAA+unr|[~Qe~`hIwjgW^Vrx`Ri[pePfIP|TTNhVnC_bmkBY\\&email=lhwhyksxjahdhuzzzisovryekvxu%40gfch&city=rtlpUGg_j&zip=62865&terms=checked',
 '/order?item=tshirts&name=os^UOAvsXIPqefJhQewKdyFq\\MiyVevUB+JF{]\\hBw\\[E}DBfrpH]aL{ZiC^Hpx[cuEDg~U+mjV\\E_vDPWdNw|LuJMuiJDrRnAKVU&email=lznpnqsehsshqivzcodgyakz%40nmpm&city=IxGzQ__CG&zip=04850&terms=checked',
 '/order?item=drills&name=rknk_C+mdzLKK~fu[sMBjnK\\HNGU`c`ZQFCHAaB{PQ[+LpSQmeOtQ]]fwRvEpi~RbArhplnSsgcd&email=kktyjseabqamrylvmkwughyrer%40ivoi&city=SAVH&zip=50486&terms=checked',
 '/order?item=lockseR&name={eXkkRU]{CDg~]\\qRpDiQNKpGK\\[\\F^NE+pMeE\\h{rasg_JqZlKx+Luk]FaidweB]iRuP_ynPM~\\^JILlM&email=awswefu%40aakn&city=wjcvz`|WH&zip=04237&terms=on',
 '/order?item=Tshirt&name=IXXQIn}arq\\WfktJhiuxSY\\EVOgU{fPq\\]NM\\+ETBUc`Smo[TEu[nPj[[Y``VttAEdhRQA+bT}z{NTpmVG&email=xpkzsfsmugwa%40ywm&city=jPxn{xg[T&zip=0316&terms=on',
 '/order?item=tshirt&name=zNsHbdZKPyJyLmmNLilW+HptpfN^yij[pOyIUfI^hv^lwhBAK+nOsPiGxh`Z`Q{GBeu&email=nyxebhlglhhjvxgrkbt%40ghjiu&city=VGAR_mrl&zip=6203&terms=off',
 '/order?item=lockset&name=Lr`rncnb}lT_SqQx\\}[xCsEmYkJszWbcxQ_+epxi^{Ux[bjHbYxxfBxkBFcCwGZ+OsJog~fht^|EINPpwc|_~ePTkxW`^lnLoHMJjnPK&email=upqxnsnnntipbljlbaygc%40jm&city=J^GSPx&zip=05325&terms=on']

easy_population_3 = ['/order?item=tshirts&name=gidT\\DZ+sfkiE{^+eLLP\\ywMTUN`[peYL]fpu}k]y^N^gT]]F&email=iwlzqtlrophapfwgnk%40dmoqc&city=ky~_rv&zip=88572&terms=off',
 '/order?item=Drill&name=lqMl[wKDIGynItJya{^^CekLNxWD_PY}zItmi{_+XQzWdBChYHt_Cjrl_hCoSUARuMeNCJd{KyPm^gcYYCVfYrE+DcAMjIqCt_X&email=ta%40nu&city=E_FVHKIWE&zip=3133&terms=off',
 '/order?item=tshirt&name=Tpz^^Y+Hlla}R`dKjnZMW_jR|s+{~SzVoGKBhmCM|Ltnu`ssgaVqetGoWuFIgt~D\\V^qZVu&email=tayacvgksixfkglpamagqobleyjfx%40fny&city=Z`&zip=24180&terms=undefined',
 '/order?item=drill&name=pkyuDvYjChAF\\vPVkt+rPIGiNQdWkZo\\zPgXFbjW+Y~CoonnMjDL&email=ynkrozlelleed%40bs&city=yz&zip=84134&terms=undefined',
 '/order?item=Drill&name=PAlZWic~s`]uwAVZe+VpURv\\+NbjGKATJe]uYyVuSR[SioV|QEjd&email=qsbtjcqwp%40xt&city=eUq_Vwv{u&zip=63666&terms=undefined',
 '/order?item=drill&name=VKNPLVq~RvWmMNS+Ip|D}rn]Bd|rrz|qs{xtgD|xHX}tjhOxMdp}{CW{n+|``iZCE}SxvpXDa[cfcHx{o\\^yAK~AKX~fgSk&email=fzxuxciqmoqmiefpzh%40xgt&city=CBKRQr{&zip=6228&terms=off',
 '/order?item=Drill&name=[_aa\\CPIiLw}WmSZLAIOjCXgtPirFzalERFkibqmLN{qc+^\\qpF\\pDY`SL[wdz_MK}tS]omC]zWzJkldaqjDNn_V`+N\\cWV&email=flvsmobmyqqpqclycxq%40gyjki&city=ePVe&zip=6131&terms=checked',
 '/order?item=Tshirt&name={FG^[uEnPJZaz_kEN|rc^UiJaPGfuDgHSxpF+YYHbeSjGfCO\\}SRN`BLtznRN`ARfyDvYZCz\\rvytRc+`i\\CwWo&email=hujnvtclucndtojklustszrpe%40jlc&city=ih&zip=4063&terms=on',
 '/order?item=tshirt&name=|GL`cs{f}N]uU|VrHiRhh~+bedudSzDuYgDIA~dYTiosmMj_ofyS+QGg{io_zpFfsVXdi_egDEGpLtJzYY&email=blbcswyrelmaffz%40mkmlb&city=RwHJeSH&zip=1342&terms=undefined',
 '/order?item=lockseR&name=Nx|rm|\\wFx|_+LHy\\War\\g`+iVoiUD{D&email=caepkhtn%40twcuk&city=Dq&zip=42010&terms=off']

easy_population_4 = ['/order?item=Tshirt&name=vNUwFBAoyAT+oZ_sm~N_E\\V[qeM`TDiGbhS`QgZYhvZVow+zgiR[BJMESzT_Q_W`tOLFoSdP|ggyouSzUGrOnvVaswceS&email=ewgwntvibdrztrdwmuthfsp%40zv&city=nBPxcVf\\&zip=20734&terms=on',
 '/order?item=tshirts&name=MwljgKvHGWLa|jt~K_+kn^fCwDhfoqcSa\\}WsBm|hmkoOwDJF}K\\T\\h^nU+RVcqfEHeQCMEf}ZGn{gcjxNX|vI|}SU&email=migbpfryy%40yiqzk&city=fl&zip=55703&terms=off',
 '/order?item=tshirt&name=M^PPvbjohBmY~lP{aXk]Hi+}efWl_UTFw[rH[Hco_qLW{+K^K}_gf[MEw^iprEyUAj_NnzcE}pr{hgeRzXyJ&email=ayscfyqtmkefnjrxrgr%40qp&city=vvGjZJ&zip=1133&terms=checked',
 '/order?item=drills&name=`qPZcXZcy_bPShz`|qoRGez_wkJmaApX~wwzpi[]]+NQ[ANDZA\\cBvAfVJ}y^UTXYmo^sf|SoKUPPv]`i+jtvTptCaFbSM`S&email=wuvqyclw%40vtflj&city=]Gs{eG&zip=6628&terms=off',
 '/order?item=lockset&name=ZGlUzW}Huwdpsu`JoFhmo~GKic]jpEOG~+BvDVsxZLA^tvJoxphFM+ueToMlNh&email=nfxm%40ot&city=DjCfLp&zip=4132&terms=checked',
 '/order?item=drill&name=uD][V+ADSY_bFc[wG^We+]qc`AakXb{aPdc}YjHuKbJ|^ctcfdsq^eOuz^DyBzjo&email=ppbmnnq%40jhdkw&city=TKXy|vJT_&zip=87376&terms=on',
 '/order?item=lockSet&name=DOMvOOQVKxLSbQ^AxOXJXrpjUgiym+BWVwgYzAOgsDTIOwS|SPbgQrICM~soqJCeOzdDI\\h+SgPZkLp]^q__VNMNI]~uVQqV^z&email=cbvduqcgsyq%40zj&city=Zk}~zA&zip=77175&terms=undefined',
 '/order?item=tshirts&name=vqPA_gOwWU`jAHi{JWwD[gX|m}[H+ogrJAbsTAp~~g_ZcqLfZKhyIqdUFfFmzPebwHfAoJ|iZNNM+IKTVzwW^oNq~VOfgpOULUEzobC|&email=hcvb%40vjc&city=Ru&zip=23406&terms=on',
 '/order?item=tshirts&name=ojbDYmz]eNe\\nAIwllU{F]fZSRM^MExQgZQtTOy_DEJbcSDn+^J]A\\^Ry`S^gnWK`trokGKExBBupZtzGNmBPIn_KwC+zx_bz&email=baaq%40dy&city=uJ&zip=0534&terms=undefined',
 '/order?item=tshirts&name=]VYBcvLR`ts[iaM{y[R]Z[ZtPnaKufwkSeg+heGEcfP+wtZLWLU&email=zsrzop%40ap&city=XsUrWw&zip=3624&terms=checked']

easy_population_5 = ['/order?item=tshirts&name=\\{`xo+fd{RCW+\\zEoMzucwXdSPtkZ&email=gpmxtubhbhygmw%40txzn&city=\\]&zip=67647&terms=checked',
 '/order?item=tshirt&name=o^^ha}HVwueplVzRbIinGbtqTXTciF|uk]hbd{C{Qobw+ZklST~iqth^RTRuE|PC^\\c]\\IuJ|r|k_aDJDqE[DUSEF`[BO}+DmzsaCRzNxygMBiHbk]A~B{``eRoO&email=ytczivbt%40ah&city=uYEe&zip=2083&terms=undefined',
 '/order?item=lockset&name=z\\GSI|KgZ\\~_DvTNYgspZO[rmaYyBZ^oeZPcLPWnzBQ`f``k+CXxkkollsRvicqS~z`asIcCzdcET+qDggE~rka`Wzj_JF\\QJEFPe~]ZtZPBXrIzliqeW`Jj&email=miuv%40usvlx&city=__ixTp&zip=4640&terms=checked',
 '/order?item=Tshirt&name=t]DpVRTManMWVqwGHfs`Hqsd+AuHwHz+nwhtTTQZdfKAHHPfHsFIzSYYjc`hfpQl&email=yzcosirztfimp%40jfd&city=lOne&zip=18244&terms=checked',
 '/order?item=tshirts&name=MMbFnySgq|tGLKd~MZhtm\\K+~EXR]FPmVDFdMVp}oFOUO{CamzmSSQOXtgrN_E\\bP+qh`{VOcbmW\\Rya}XY}|oP\\x~qSb\\^y^&email=ohttbobdghisjhbnaoopd%40oz&city=BnO&zip=2311&terms=on',
 '/order?item=drills&name=Gfvao}sPavAfMiIhL{}L+mY~WY_aXJRM]zP}K]ntNaiLrwTO+V``agcopDYJg&email=uxbvyrepotecwegxujcy%40yjd&city=J~q&zip=6622&terms=checked',
 '/order?item=lockseR&name=qbWhYW^aD^zGBrmY}PXAo~+Ts`c_T^lRKBT\\EaXAdjJBhMLEI+JjeR}dLS[YA]eCCo&email=hvnuga%40dyc&city=XpxbLs}`&zip=6858&terms=undefined',
 '/order?item=drills&name=LemXBCgNtv`x|{sDk|Du_GTh+o`NQQG`u^+l{D}IYjjo|sDNmbLM\\]ruPRM`u}rGrAODZJsKDx&email=rkehtvf%40iisc&city=cuwoGet\\Z&zip=8517&terms=undefined',
 '/order?item=Tshirt&name=tqQgwKZ[rungdsR\\P~h\\|~]`f`Z]_DXVsmUOdKODlzvxr[_+TL`KQjRW{EtuOw|tIcrWWZ`GSRLRxPmQcp`]H+xF_arunUJVpbyod_p]K&email=mfnynrv%40zk&city=WxtBs~&zip=82235&terms=off',
 '/order?item=tshirt&name=~HCCsCz`qQ^uMesXDoAP^jlpgKI+g{HW`rofTB+t_aaAVtn\\RdVLhuRBjlu^t}\\YtqvONdgDXAPCCgKwhRXYK~Es&email=qdceqzurjcwvrbtuy%40mstg&city=GOkCCpdT&zip=17580&terms=checked']



medium_population_1 = ['/order?item=lockseR&name=ldpNi&email=D|YGFOGtogNJXFK`&city=qxwttImFDA&zip=5LaJ1&terms=OfE}',
 '/order?item=tshirt&name=E\\RZOi{BHhZC~W|`TmD[{^nAADCVqCa~lEAS&email=[leUKirshBP\\[btA[~S&city=|BClYiQF_ldAcFJwdxZk&zip=aN98A&terms=OIOE{',
 '/order?item=7rill&name=|CLuljs_AHbXqSrrcm&email=u[VJuRreRS~ZZ`nu}ixa&city=&zip=QvpMa&terms=GhWt',
 '/order?item=Tshirt&name=POZnyDrtP^qQJIxDUuCBUr`gweG|uxRaYfgGCYkkDN&email=cMVjynq`&city=O&zip=2mHHQ&terms=ppZnY',
 '/order?item=Drill&name=RaDh~H`uzXEG\\jxBdj&email=DNcUGOGxNKsDJ&city=lvD]pnXRwOi&zip=B0tp4&terms=k[g',
 '/order?item=dr5ill&name=giEzDxb[oJ|\\TEgQvDoMJoZRvZWFjBxKCu&email=GQwhn|H}MNqqGU}&city=dV&zip=ZX6ts&terms=PwufW',
 '/order?item=drills&name=n}`tUgxp~xK[OJWHrHHArtxypa{CerHW\\~Bf&email=yZp`qkG^NKpY&city=lNzpyAX&zip=nbGHm&terms=OVRVZ',
 '/order?item=dr5ill&name=CFZNHhnY^UlLsBCBUOcub^cRwm[\\b]pezvhO~&email=]Nafq&city=FBApNOJ&zip=WUKOn&terms={z~NJ',
 '/order?item=dr5ill&name=n]LNlBend}GhwBN~bNAvgd[FZNclFj&email=PZhEybs_xFmWS[`iGL&city=dc&zip=7SriE&terms=goDGd',
 '/order?item=lockset&name=CvKuqbzGeIxQ_ZhwQi~qRpQH&email=mrtc\\WIsZOvugQWINK[mBsse&city=EiDZfTsBE\\tgZcYHRSarQQj&zip=7fK4t&terms=xGIe']

medium_population_2 = ['/order?item=lockset&name=phJmt^NXl|fzbw]xy[cK&email=KuEueYuq&city=kGJ\\QY^SBiD~NmrHkZR&zip=etXBo&terms=}kr',
 '/order?item=trills&name=|dZoN|SyWQ{nUnQhoc&email=i}gCqQ|rCCh~&city=fDG[nDhR~XwqJ&zip=3mj82&terms=qh}Fu',
 '/order?item=lockSet&name=BNSXtQuNvvm_CfubwPZNjes~DbKDZFWgNT&email=\\oTIJFk~BQbZ&city=FAX^tYOPuWxsNKnpTHxsB&zip=m7ufD&terms=~yS',
 '/order?item=drills&name=`fyZTMiqRTICENOqIwcxSBHv{eTXFEG_F_Eaz&email=IgeL[zQCJuAr}j[[bK&city=Bj&zip=xlyb3&terms=Bheyq',
 '/order?item=lockSet&name=ONyMoRaUK&email=qA&city=QLVsvEdeKc]DjC&zip=jtKPs&terms=MpPt',
 '/order?item=lockseR&name=jTs{tkxTxUUKyQ_BY`VIDutbgOmKjtnPPRVoHJyMg&email=L\\dhgI{]YGkkMHIganjl}yoaHM&city=x{heToriUuejhqxOP&zip=jLvz8&terms=BQbSf',
 '/order?item=trills&name=MJ_|Os~{UiQ^wY`pmQTRh&email=gtmjlU]IFDDfJgQuz^R}YVk&city=j^BRZ&zip=iNMhZ&terms=Nmso',
 '/order?item=7rill&name=tAP][QLqYsTWoCM&email=CbioMFmAU&city=PUFBao\\e|FUcS&zip=gUj1h&terms=_tC',
 '/order?item=PLockset&name=gMDibjCZsSXtpv\\rXERmkL`~bsK_Bc^lGzT[&email=^\\v`UKIhApPp\\TW^}&city=qXDXARUs~C[DLnSpvvmdGbi&zip=Z1w2Q&terms=h_nU',
 '/order?item=P0ckSet&name=EMo\\NcLZVTcCLha~G]Fwg|dtMNqodrkXWzzYNzD{i|&email=F|~HXUCKmT}hpq^tHSAU^&city=i]gfEj]^h]CcbmdsaO&zip=EOj8n&terms=|A[']

medium_population_3 = ['/order?item=PLockset&name=mVDDVW{C&email=ZPR`sq|&city=RW~GpprnZaZLM^z&zip=kPhcs&terms=OXON',
 '/order?item=drills&name=uSoBG}GOFxSI&email=vcsAf\\MFvEcOt`IZdqWXMhfe[D&city=h}_oovisBVG&zip=grPQu&terms=rMh',
 '/order?item=Tshirt&name=i{}obHPGEghLdhWeaz]p{DcNT`baSrcxHacEkIjMDSi^d_At&email=ITeXX^u[WMFoXtQtkK\\DoDc&city=KPxkKK&zip=N5TvJ&terms=xEeo',
 '/order?item=lockseR&name=fdf~Y\\SmAvq^hC&email=GEj&city=EQc&zip=WxCJ7&terms=Rii',
 '/order?item=lockseR&name=ddZndmKkghOtVtxGJRJlJa|nr^&email=d}kTuPpKt^SU]aCeNs&city=fOpakV&zip=UQjiK&terms=k^knt',
 '/order?item=lockSet&name=Atq[x|bn`NPfmHJe|AgOH|mAF&email=dfd|&city=EhoxYD&zip=lZf4S&terms=Q\\U',
 '/order?item=lickseR&name=jF~aSVaSRUHZKo&email=BiU|E]K&city=g[gvVj&zip=9Y9Sn&terms=XPcjG',
 '/order?item=dr5ill&name=\\jwvJHLQJA_fub\\o&email=g`[JNHBGTa_^LbP}z&city=c}\\sqgKO`&zip=KJXPE&terms=|hQTN',
 '/order?item=PLockset&name=ikq__nuVd[T|Q&email=DodfHufVEwgnoAKQ&city=e&zip=YU1Wb&terms=xfuww',
 '/order?item=lockseR&name=TCUpHm|uPMsIzyuSvkffws|pK^gH_`DIFtM~]Wb&email=H~zBtcj|GCkxkiYF~T&city=le^e&zip=t9Okl&terms=ewxls']

medium_population_4 = ['/order?item=lockSet&name=QZFGZSySgB~FZeFwV[nYWxuIDlL^jM&email=SOMPtFEUgS{VdeIG\\vLYpk}SbJ}m&city=`GAEaLXGRui&zip=EPfoN&terms=\\F{Y',
 '/order?item=lockSet&name=cSXiF~vcWGvKi|h~EakWCtuM}GhLTWvFpa]wn&email=~aswnHUrNTTQH|Jo|^mOKR&city=kr`P^Dn|W&zip=wdW4O&terms=xfQ_d',
 '/order?item=lockset&name=AHdSG^adQJzWrGvdTWd`kOJHkErbW]gEdWmB[qSGZAGD&email=rI&city=zESlhq~xTN&zip=MeBMS&terms=MLVo',
 '/order?item=PLockset&name=p__VJf{kiB^[yQFfqfOH]FySTYvExAJgfmAGKDSemC_JcjLW&email=oG}u^NUknIYb[yWj|WlW&city=jpX[atchhCd_FEL&zip=4avtl&terms=xHRAs',
 '/order?item=P0ckSet&name=Hrnb|YgzDSYRWNoRsLZ_AKvfmJNMFOEPXLcb&email=vHD]_TkPFF`dnUhQ}ocM[Pj&city=&zip=DRSLb&terms=kpDHy',
 '/order?item=lockSet&name=tvtDegQ[J\\pPe&email=MbNji~|He&city=vDfoX^CBnhX_n&zip=Ms6GP&terms=Uf`FF',
 '/order?item=lockseR&name=ZtSZ}ErZnGB^B]RiFadlukfNEn{aGyRbtPajQxg]W&email=YRWOxcms&city=PnHDgJr{q|m&zip=UzxEX&terms=exzWt',
 '/order?item=drills&name=NtTL[YK~f^_P}qwZVip&email=plmK&city=yF}Ig}o~krRUs}[RwOzk&zip=MKY7g&terms=bcOc',
 '/order?item=7rill&name=HoLLgQQtngM^mZbmBgUx&email=~VpPBo&city=j&zip=FEOQ6&terms=Cqet',
 '/order?item=PLockset&name=WMJcoPILfjuVWJXrHEIQdBwbEO&email=fnSePclqrgDtZmNwlZopnE~K&city=lLwtr_&zip=R0tVR&terms=olvoJ']

medium_population_5 = ['/order?item=lockseR&name=NUzyZb^lV~v[iXdD`]WcfcVt&email=ZHUr&city=~WlfJK}yiPiWGf^ut&zip=OYyJm&terms=YBzoB',
 '/order?item=drill&name=}Yu^]{ZQHEcC`prPveIzstv{gBHUOD^&email=}ZLXRHMDJApxP]{TrdbiXLwk&city=|NC{du\\De~}MoPwED&zip=Foz86&terms=WVl',
 '/order?item=lockSet&name=QIWt]WvvkxWSkoNXl[T~__Rmjl{enPnoQaKdg\\XXrZ^OdF{C&email=R}z\\qNWhr|r|{zE\\j|h&city=bQVo`XolVHpOCAXRad\\&zip=zbvHI&terms=qCAd}',
 '/order?item=7rill&name=iQwa}_khox`}yWrtkAwt\\AsuYubV&email=yoIBPLSRmpKf]yJmD|G&city=~XbsIDj&zip=jG2gd&terms=Eqk[',
 '/order?item=7rill&name=s~LmWN&email=bQ&city=}vYpUIoFHBtl}ay&zip=peBZx&terms=Igzy',
 '/order?item=7rill&name=OuV_tLRvCM`elGAku&email=|fxk`LhY^ziaWmniYiInooS&city={PaiJQFh{DLapWBXUib|VY&zip=C3pFM&terms=^EG',
 '/order?item=PLockset&name=naCf~[`oAk]JToypvUoHlKWWF&email=Ix|UcwATCXEPrP[UaaRMD`ELrsD&city=W{fZsrkive]&zip=XNrv6&terms=T\\n',
 '/order?item=lickseR&name=E^UWDSUiMYeLI_hi~]&email=ska^QoeJiDGnJjTo&city=r\\U}j|I\\MYrQxoH&zip=mEc2g&terms=Sj{',
 '/order?item=drills&name=e_Q{hvnT~MxzUgwDGouUDN}aLJ^`ffSwbOiYWXzOLXVxa&email=V]XZFwci`mBkLgnYtMEj&city=Ruz{ZImmg[\\~qY~Dqkz&zip=lik9X&terms=SOX',
 '/order?item=Tshirt&name=`iZda]ntlUyWKyHZ&email=Ec]qdRA[OyVdWywHq|&city=[`Zj\\}Pwn|]CTRHt&zip=9QOVr&terms=D_NV']




hard_population_1 = ['/order?item=kgEtxX]T&name=奝Ũ㽽\uf571翫嶼\udcc8伏ല㒌祄㍨軾䟠黁캎ꋃ&email=剅渽㸾羫ꦌ녿鍯穷烌䌏㷛\ue12a脓옐\u17eb\udad1\ue46a㌺\ueab1\ue484醍ᕮዂ鱡좣\udb1b煪&city=gGFxPeD}QyJgEUOkB&zip=kTlbJoXG&terms=lSP',
 '/order?item=mtTP&name=滧쌧ᥤ櫫啕\ue1bc쾇甓蒊\udb8d䣳Ђྨ쩒\ue372⣼\udcfcᩧ\u191f൳ꄄ⺟ꊨ뱸ᄪ&email=⠊ዢꛅ옋ﻘ얎\uec66픤旱⺨滾\ueacf컞\uf666柳ᒕ暴篻⼸弨ﺩ죺ꔏ\uf877頒嵤ᮘ㩟&city=ofSEfycNuznG&zip=1jZY46gY&terms=LhpiQ',
 '/order?item=wIXu&name=焲㩪\uf4e8♊忣ἢ壜ﯜ냪ﶗ\uda56ｲ璶\uf48fṓ꜌ꃻ挰擆ሾ别ᤪ艝\ue8df橙巐깩Ҹ둾\ue3ab格뫑\ue865机沠濔溙\ueedd倐뿚ﶢ&email=㡤糣&city=BVVeh^gqUmtweUN`KE&zip=N5NlXGT5&terms=PqN',
 '/order?item=jgEpuB`Q&name=㳌㍐쎍\uf12e\ue6c6\ufbc7觕遫ષ䖚聶՛㚌㪼ﰫ⎊풐ݫ椛ꑙ묠뾞穗逕歀맩\ueba9&email=긴\ue78f샦ꧼ⠷\udc8c괞陀\u202dᶢ\ued5bᏼ㡺\u0ff3矙ꄈ췞㑀ח톧洿㳑ӿ\u139c&city=q_&zip=cLoPXNMf&terms=fR^}m',
 '/order?item=twotiv\\&name=ﾨꝕ寮§悦粠釶略侔돴ꏛ踦ɏ厝莠콁ᡖ皤閕⽷沺ᘂཧ﮾赽灤餖\uf664ኪ葎꧱굣蟲旦浅疑秲፦\uf714ย\ue418꽫\uf069\uf16d砰밥&email=\uf433\ue55a⸎⟳쉢妑葐\ua7c2ᵅ莜豺㴕\uf60b뤍䭻誆䇐&city=uuT\\zWBk}ZddtsJ&zip=wJsk2LQr&terms=NHcZ',
 '/order?item=Tnp[}_SYv&name=趏榹꿢봢퓩屇쌜\u1caeႇ凧\uf621ɯ飓獐ퟔ阥⠍㏣歗&email=ꅀ\uf0e8\uea34挥먉\uf656纝쐰\ue9e7&city=\\Ms&zip=6uMk5gkN&terms=ZXa_',
 '/order?item={iAe&name=毙ᐆ咍喸ᡒ&email=\u0eedਵ궩⳦ꞅ◍═\u20c4푳믢얛ʸ\ueb56穝&city=kKw`XOuDYfMyAqAM]&zip=QUcyIxtb&terms=lWjLq',
 '/order?item=Oq~&name=崯꼟\uf561\uef97鱕\uf8eaꃏу㵧\ue238酾&email=鎉\ue30f\u0b84ᯄⶨ楏왅̒ᔸ謩迱粋覯ද甄Ӥ쑝窟鈿\uf405䞷ᗳ짊ᵟ䘆ⷺ&city=bdwNcZmfaafq&zip=RqtUcIz3&terms=AeOY\\',
 '/order?item=VUy]L\\ol]&name=趘\udf7f䍗\u0383ᓾ콤轝閞宅坎餤☹ᝥ듕&email=͗ꎴ鱝\udd14\u9fe5鏍⅚羛꼀橖首\udef4\udd1d\ueb51㸤\ua83b詃狾Չ&city=QlPcvQXu_hwshIEmm&zip=BKhN6cdu&terms=l^dd',
 '/order?item=qvtzmfR&name=ԃ᳣㤇촹綌&email=䗸\uda41긍燆⌍뀤㎽鑼舴仟\uec6e&city=P}wE&zip=TLJN6uV3&terms=fnZ']

hard_population_2 = ['/order?item=fnCaU]&name=䥊ພ\uedac澏懺牮\udad1\ueb39钞梃튠塉댺毜✞獝✢戓\uf265䩤㡫\ud9d8拀㱯ᒐᏑ樚岩鹹ⲧ\ue0b9翊\uf357ܗꠧ\ue88a\ue412\uec50\uf55b&email=\ue86e\ue007\uf367\ue838ᎌ쳸&city=Xt\\rLIrRshk&zip=iEf7zY3z&terms=Bq{ln',
 '/order?item=yfr&name=৳엍잓鰔똞⏜蕭鈌끹脏偈穻䱡ᵨ蹫謴\udc0e舼\uf5a6ꯡ햳⥄䕲⩿냐⢋韴睙䎫㬱矨䉡ⵙﾸॗ㑰雧潙◨ⰳ췲阢\uf0ef\uea93ࢲ돪&email=䑖歜\udeb1㍕匧娇⤸摮\ueac6搨獏&city=|JoxX`_FNEPQQF|&zip=OMICjXza&terms=h~bkq',
 '/order?item={gnl&name=ﾛ嵘퀻櫖땒姴ꫝኢ휥璇\uddde聿漁⩑骚ᣬ\uf179⢷\ue516\ue577ᬕ㫥緰欑킺斂沨岚䤵냊\uefd5ᾣ\ue743খ༱陋瓴쑟吿ᦋよ冣Ẅ㥭焰硧&email=ᘫ웝\ue18c祰鉬䅛ไ牉璳︁ﾖꔃ┶螄䚘㛓ӳ༹䧮⡑욦\uf3aaﯺ㖈얳뀽﹛&city=\\MXi~MgtKH\\mL[LSz~zZvdE&zip=X1wb6QAK&terms=w{R',
 '/order?item=EGIQHVvR&name=ᅱ쉿켧뿱圐&email=ᶓ듻\uf693祙뜨魽\uf010쒭놀&city=NQXxMhSxTwJox}^`^h&zip=L7KqcEvB&terms=uDA',
 '/order?item=qm_m&name=〶䕼佖蝩슲\udcf8쬦씸弣ⓥᷴ귲\u173d憁澸\ue610뫱콀⓾넃儦겿宒씔&email=\u0c8d鰯鴞炼&city=DWsJuWEZpifPAclrR&zip=z7Mooc8n&terms=PoPl',
 '/order?item=SahNIcnBe&name=\uf18f⎚ᾦ䜦怖䃘\ue19d䟏엕茥&email=쒠ￅ藽赧팻鋘脒靊䘐鄷跙骝띡&city={{~FVj^xrdR&zip=4sqU96Rq&terms=BbTVd',
 '/order?item=XJ\\X&name=焚Ⴔ泏\u0c5c齺뾼פּ嗲㦊䱋￫䵥樆ç詇ﱇᖎꧧ㫦\uef67鉰뵙ⵍ梮넭&email=촢蒓&city=kbybtNJVGY^&zip=ODLSSuA2&terms={q[',
 '/order?item=D]qO&name=᪁ﰶ酝右鮨嗜\ue2c8载澆潿᩻猢鼇㔗\ueebf轇퀛톿ᙥ㭍䝑关㟅谩檤逥&email=\uef9d\ud97c＿&city=qUNlQdoCgX_zjjeKig&zip=tgZdzX0g&terms=LCNh',
 '/order?item=nXbxr&name=䑖븵澗\u05ce魍≊\u0de5\uf60c蜽諸ⰷᶎ\u0fe2莲\uf4a9惌䣆\ue5b5\ueac9\ue224觫ꝅ翉胏&email=坯ẟ缾\u2bfa䭕㫶똼ࡆ\ue6b8놇豘징ꔐ뎲\uefa3縁蔦⃗&city=PPhUSaSrwe}{vicR&zip=MqnwiuJS&terms=wuCo',
 '/order?item=e|pX&name=\u05ceƗ䲶协艼⸂蛍ྲྀᮎ\uee9a轐롩ᒞ邗\udfb0\ue0caᔞ䫞婢笸\udd20覈衍禯\uede6ꁋ躗Ս쩺躮ﺢ潩绅ּ甌쩊&email=旍\uecbe肛湁ģ칾ᰈᴵ遽뾾霛깄ᇹ踸혔끣囘Չ分鼬斮Ϫ&city=qY[ahztvk&zip=oRAlKBlF&terms=`mPd']

hard_population_3 = ['/order?item=hIGC&name=﮳\u0382犸竖穖%ᬌ탱枖力鸦ꉰ掑穃䵬枡䮞\u171d됮&email=ꡊع\ue215䵖짱괒&city=E~uoB\\nGftJCk_ch|aAU&zip=tHx7HOmR&terms=gPdCm',
 '/order?item=mL|z`RlI&name=嘷삨\ueb15㽵岳蝠벧ᗫ䪭ଈ髫\u0fe2䯯矱쳯稖퐝꺥\udd96엛㱕ꠀ潡\uee41軼ᬳ柛ᇕঋ\udf9c㌏㗡㶄쵁줡\ud93b\ueeeb&email=벵ᑨ⁐㟪͘䊹❀ⰿ簎ၡ陊䕾䁆䒐頷٩념&city=mQJyv}G~s_EPY&zip=aJr4aEKi&terms=ItQQv',
 '/order?item=fUv&name=㠐憦䵺엂\u0c4f㰫㉶\ue8fa㲻\uf481\uf8ab㒱⼀뒳\udc58✊嵉찁됲䗪鲵짯\ue565룙࠽綇깚▉\uf080\ud8c0\ud8ff닦ꈬ&email=ⷃ賞鈴컀⋫棣蒝餻㾌鵿貙桥ẓ\uef0f﨩霰&city=sX~d|YlnYQ&zip=B4seZEoF&terms=CLUkB',
 '/order?item=eO~xIo&name=爫凴潙蘰\udc66ሆՊ攋ɤ\ue4fb蓒ᕻᜅ멭ꅏ版甠䟱还挅뿆잂\ufa6f\ue1d8鎛\ue27d죳髑哽숅쇵짞\udbe8뒏雒Β⩜䆿ꚉ괠탺ᷦ葞趲ʦ럠&email=ﶢ踅㨷瓭枍屷\ue0f4绚⣌ஶ䋈\ud80d源诔䔶譹쓹둮똣綑㭰⎾&city=i~l]YKdEkNsZbnMYuNfTO&zip=ELEfisYm&terms={QYvu',
 '/order?item=Wrx^nYved&name=뷤䡶ⴾﺠ\ud95dꩀⰓ\ue040⾾䘅\ue490邲錇爍픀㠸列Ĵ\uf4f1髞৪㴴䴌縨뱰昚แⳔ鏡\uf2d8畗&email=᥀秕\ud8f3蜻&city=xdQGCHOgilXv[JI&zip=pT7K6XYm&terms=ZCt}E',
 '/order?item=Ja_Od&name=⛻筚ᏀɁ弞ﵜ\uef2f皀㧢嬛ﶷ칈\uf275鵍ﴮᓬ殈脎휅\uf5bd玪磺&email=恦䕩\uef85ꦫ\u1979頕몇ଳ뗷튮轀\udcd9瞦㍲썀ᤡ깐彗&city=ZGwDGZxsyGwj|qgme`bkHY&zip=Hw7LwPZZ&terms=CQRq',
 '/order?item=T\\Vn&name=尌縗暹⠣վ\uf5c2⧻큛璺㋉瘁셀䞂ᜂ嬝᱕誠&email=민鳙ᷣ练ₔ☑㉋ம䀧ⶸ\uf5c8刑ؿ䂺⾥솁酆ꭳ✿㡲\uf4f9\ueaa4&city={FIKiMEeq&zip=hkjrR6fq&terms=tum',
 '/order?item=~qAUx^O}q&name=\uf1d9\uf00c틗ሠ菴\uefd2\ue902腙䔞Ϩﳰ\uf089ਙ퓚Ƅ勇\ue859\ue69bᗜᗉฅﹲ뙰녈˜톮॓輵ᗫ대注逶\uec56\u245f퐟鹶揢\udabe쩚怍黷&email=帡ៜ\ue11b둌&city=nDS\\Bb|Xoh&zip=jMQ6kMNe&terms=zt]EL',
 '/order?item=^IEwv\\D&name=톤㊤\uf2e9壬挽鱠缾跥ꯌ砺醆ꔸ㈳囤诧ᦣᡵ亞ꘌ❣㧛ῷ祻钘韩쌵绐\uea3c\uf6bc\uec3b멨偣颿낶쎎敳偋갪&email=췱ퟘ聱膷極\udd1aퟷ璎\uf3feឋ⼧&city=unAPJZ~MGD_{&zip=wB5uoWDJ&terms=ETpc',
 '/order?item=U\\|]^C&name=Ƶ\ue4c0\ue599ᰢ왇爛◚\ue0c2כּ衐沚㛼\ude38㾨䦹䵃ȇ隕ꐵ&email=ꐂ韺䊆劘兮斓霠愡ɰ倄촸៦貘䄟\udb49\uf065䳝悫쉵&city=oeUdHmeeAPNR{UnQ^&zip=qNoy9kHG&terms=Ikh']

hard_population_4 = ['/order?item=ufr&name=浊\ue7a6反ꄞ蜾ᴹ翿\udd73犄䟟諕㮱츫ᱩ鰤嗓哯괈ᎀ庾怇謄䶯畬\uf79a\uda03턼苊뇞ঽ┟餆&email=\u169dᤓ쀐蹝棉崧䷣귻ሄ縬솮\uf352헣妋∞ꐱ쟒ㆉ銋\udaad&city=ZrgbXZ&zip=Qdl4YAQE&terms=rvJG',
 '/order?item=qLPR}BG&name=円尀ⅾ译兠覭&email=잎⨱⭙⅀歄婋㸼ऑ嫎&city=jyI]_&zip=A5oyGmQO&terms=fey',
 '/order?item=eLkRkZ&name=ﮠ読\ufbcd\uf098\u1ae1Դ\uf3ef쌚ᯨＤ黅襤胒탃牛&email=〦\u2cf8翤辝勞浝䪵發๔㛒᜔å\udc55\uddd1춥룔摠\u0e86羱롶쩱쳮\ueb4aỵ鳯ꔪ並ޤ&city=wY_gjw&zip=sGQzukZl&terms=hwLv',
 '/order?item=LA]WVLm&name=嫧䑤屑븤읨ẕ叉&email=☊Ủᢼ䫎倭얲㰯Ꮠ㎈\uec90\ufe67꺘髒꣫&city=Uu\\^~\\\\baH_{P{ey`e&zip=oxTJGr2n&terms=HiuKC',
 '/order?item=^aB[&name=ﾑ䋞豌稟䘔왲\uece5㒎㳩弨翞\u1aeb좢\uf034ꭡ筞㵪\ue36e鮶髛㭿諀糖髂蘔쿰Ϛ\ue501祉虽䴒礡ᬹᳱ솇뽓&email=㶄\uf760⻓愈ￖ㨡\uec02\ude24強蜃퇍&city=eq_g\\S&zip=1CXtlNfa&terms=ntPl',
 '/order?item=yuiU&name=徽鱖\uf558䎷뻩ᘏ䋂ኯ⚍췽䚌᧾䧤\uee1d䭓鵊咫\uf540㽃㡐榼ᕗ蓈⇇䖞鮹\udb77쪛&email=̳聊뀧싍뎅鱅倃\ud859嚪䚴ᡴ&city=JVWe__c&zip=3UtCAe8e&terms=F`VY',
 '/order?item=y^rFn&name=⥅摥♋\ud851過꧋籴范\uebdb\ua7c9苫쵢쭝छ밎槤〨접䣈伶迉ⱕ㵅펓铈팲郞\u0bc4ﴲ鹙ꀬ媳&email=팊ﺲ疮㐂擒ᝰˍㅀ夆菋橝뽯\uf49d餤痩掞\udce0—&city=rbgErYkP}w~CZwq\\TYasN&zip=1oKMRVfE&terms=lka',
 '/order?item=i|CyRgZ&name=㝢䃏䴏怏햺蔳ⴇ똯乢ꇄ腎魕㔚㭈珀莻威\uef63䣦뱄ꢿಬ&email=\uea35㊷ꖢ\udfc3\ue676&city=IyG\\{Vb&zip=VdLbVH3j&terms=Qdqu',
 '/order?item=Olg|[Yx&name=뛁彂괒֟憌ᆂ詜쥃濷㏀︎䥱漰氭쌇࿁柼觵载ᴦ갮ꂊ䒼ᑷ\uf54c鉈鯂惖䂧袱萸\uf518ꆡ웊\uf86d⟵\uf4cdಐ╻泲鈼陭띇&email=﨑韴狯\u20c3ඖ繴ă儸뀨㳉唳誌투㟕华㭸楖왚紴њ&city=IwkogMzWuAqoVCvUraD~Jb`&zip=d4Ol0E9S&terms=wtVC',
 '/order?item=ANXJ{jAK&name=曯\ue43fڄ촅掽ड़휟ᖦ\u085d剢쯦㭧갩\uec59仗듌惠\uea6a馜&email=뾉תּ苁谍굡茁⻈㸯美⺩ᢴ삄&city=Suog^&zip=L1yt7Dfo&terms=rA}Se']

hard_population_5 = ['/order?item=xQjaSnMM&name=ꆡ⢊훿선ឲ椉圬瘛昙䇆ῩƄ昊몡ꅕ癸磅&email=蒏\ue3e8艆榖ﾔࡈ\u0b0d&city=}Xbzv[WizSW~u&zip=1Eojkeem&terms=CM{ZQ',
 '/order?item=ywLI&name=\ue997鶻ꆂ㴊ꓝ⋷⺧㙮壖ꈑ䅸爢柏ꀕ죗ꑇ穎&email=\ue50f苡ݸ爬樯鏪ᙅ&city=[{wq^zfGUdSyqcJBl&zip=PoKd9ENE&terms=qXj',
 '/order?item=va~PeCT}&name=ٵﶗ㺤ﱨ﹉㩎椵ꪉ俬铡缦놶촟\ufdcf먉쟬跰蛷⩻慰磉暇䖾쮺፞珤\ue92a溛彦ꔉ&email=\ued90ࢱ㹧ປ砹䳎\uee11&city=}HPocs_LcnS|M`bn&zip=63ZcAweU&terms=L_]G',
 '/order?item=xewB^spR]&name=廙\udb2bꔔ伔ẹ\uf4d6⏐웫\u0e68ᐣ수\udc49䘊铉潐秇윽一\u1ae4쑺ळꙜ㤸炭\ue6cb煭銋톒犍䀞거울\uee28ꖆᅢ䁒&email=豊ḳ缢紒䢕\uea3eා〟⠽䯥檈ꗄ\uf009䨼䓢ㅃ㰗Ꝏ๗ꐯ㨃舎\ude21&city=Oti_vld`^tl{FHIE^tIWR&zip=Qoi2P3Zu&terms=xvEoc',
 '/order?item=EQM]Yaa&name=\udee6ॠ椔䟠б\u0efd쓔쯀䭴ᐄ\ue7faỮ\uecdc\u05fe텻┥ⰹﰝ滮燡軼躔ᆧ\uf4ed䏽䨋鬭&email=\ue15d萡牞較뛌߁‘殎벨굢㉩샮憒秓ྖ턼ᄎੳ䅁㶽踗曍뗯쒠꾷\udd53魕鎼㯼&city=aZjo^dkNkmUdlxCrLxWOYRvk&zip=4eU323Po&terms=fzp[',
 '/order?item=dMPXYKiTb&name=뻐➁鋱㠐띄퀠쎮&email=\ud919鸍\uf2f5⳻뀳ꥣ\u0bda卶\uf224೬\ue6e5쩞椿퍭렴&city=t{|cqJN}~FuaiOP{s&zip=keFuprwW&terms=R^VQ',
 '/order?item=gJ|RBmK|]&name=瓠簇鱅䂃㊱䆸⇯\u193f\u0eec魷ꛕꬒᷞ\ua879퉍㩨攁滧跏믖&email=\x1e糧췲&city=Eu\\cudkUww&zip=O1KGcaYc&terms=V^Je',
 '/order?item=fMHaMVsiT&name=詤\uda9bᮎ\udd0a僅쁊쁡\ue0c7䩶쉿᷌띤⇐Ⴙ沀ﯫ험篗&email=ೆ郗㣴㳿ፇ\udf25∓ᱍ줬&city=CwkYxzeIVH|RL&zip=7tRSF6PE&terms=\\Y~',
 '/order?item=OrUpP&name=蠍檒\ue1f7′\ueb65駭䀆\u1f7f篹춣䍪\ud8c5濞㧹圷ꞗ㲡䀫陴멢\uf79c吘쭘\u2beaﳡ椲\u2451쀠际ｾ櫶ҡ\u2bfc헸\ue0ea䅑\udc80鱈옷&email=힁狪訪&city=TG{E}Y`YSasSuYVJa&zip=DMuDN2tD&terms=Ytu',
 '/order?item=FGDth&name=ꮎ럣鵤襕䞛\ue938⍿\udc0b\uf6f0镰酃屪뿗㋀㮮樤镫묛\u1ad3汌瓔憶㟧\uab10◈뒝ᑀ薈爘ʆﭨᖾﻒ뒰&email=䗴楉鯵嬥漂\ue456\uded5邘\uf5a9\uedcf\ue861企滊䱦ў㚹씄徑\u0ee5\ua4cc푗嵍ꌃ署덍銺큳潶\ua63f&city=cccQupDCHS&zip=fjQMEpHz&terms=n^RO']

complex_population_1 = ['/order?item=\uf87b\ue310\udbc6㊋姻䏙朶&name=\udc52翇㹧㣂\ueff1А\ud8cb\u2001됕䃍꼧\uf478䭚펂쑐ㄒᙰ禳द愩&email=\u2e66흽썯\uddf6&city=揎&zip=*xNoUh)P\\h&terms=ﴚ슡韼쳜',
 '/order?item=㑩ﱚ⢃&name=髪ᕓꎛ沠達卖氾ੱ낛샒呞䬉\u09fd堳鳍祡渎\ue956津ᶙ쌆\uf4ed碫磀\uf20bຶ&email=␕텙\ue112\udb86⌵옼ꯒᢽ긂楩\ue542當\ue8ff倪酃&city=磫剾ር\uf8f4瑑䄿锭냝賲\ua7ba绽\ue697䚒\udb34潦飝폁&zip=p*#s<g7`c[&terms=কꜝ皯॰ꀊ',
 '/order?item=碐繝&name=\ue129쓽\u1af2ɯ럓坪욂륗\u0df6蔾䫮墸躐\ue23c팡ᚺ齾\u2fdb\ufe6c\uebc1\uf5c8\uee59邚ᔪ\x18\uebd2鲲\uefbb&email=皝卷ﴽ탛甏ӗᾰ\uea2a⻚₰蒳ᶜ둕⒅\ue863ⷤ퐑藢鮰䖛ジ帒\uf458偆搄&city=ꔦړ\ue3b9砻羊\uf54e橞䟿壤⸅勎뭜ݕꟸ&zip=a*Sa`t1I,8&terms=',
 '/order?item=㞼\u1c89坃帀㊬퇘䨣阾\uf29c&name=䧯ꚼ$ᵭ䘕뮊곅紭籟呙彊&email=\ueffa㌎᱅胏☕쌬礠桸홒ꬳ軦⪶ᵕ&city=ꃆ㟚뼶Ἳ\ue237\ude9bᅉǙ礪쾎숩&zip=zv+V%40}3~&terms=润佢',
 '/order?item=퀘&name=&email=\ud98b儝\uda31틬\x95㬎巿猔ᒡ狆鴥\uefb6⼹笰䮭\udbf8ꋒ搜ꭄ♅뾳摈弔\u0bd5눁뒑斀螴&city=櫱뎛⒤諠\ud9f1뢽ꩊ袗毮饄⅃秤؆&zip=9<q38qI${M&terms=펭\uebfdꈝ\ue230',
 '/order?item=&name=♷婧佐섒敖Ꜧ钤\ue0ce&email=\ua8c8\u0e67粀㲪\ue2ce㤝뒼泙\uf1ed뾬鐬䪥ऩ\udc03━ᮈ⏀㌙\ue76f僇㏰瓞&city=铖遁ᆞ䮱〵㐗&zip=EcTY)@c`b;&terms=Ḍ㻲큞䩴匹',
 '/order?item=&name=ﻮ\ud986\ue930麰卝极쏷껁本㸟륐犿⇍䠸쐯釄㑁ﲦ䋚棲檵菋\ue5bdグ죎\uf7db囱頗\ue1ba&email=뫦吪ⲉ&city=献젔爭蓀骘鷟ྀ&zip=L60|*,}=&`&terms=',
 '/order?item=莕\ue37f璨搇ꢊ&name=赀皝㙟逛ֆ䇡잠㈰둁ₒ\ude2b饟身縲狅︿ኦΩ≥䤸᰿\uf30a笑磈⫂誻\udc85滽\r㧵뙿䟮⧣䁀퇧\ue50a\uea93&email=\ude56ꓝ⍂\u2d77ꎈ郪ꈄꔾ话䤄김蘿\ue29bᓔ\ue652貰&city=馈ꔶ繇酽\ue267\u0f48⥚䢤\u193eꉐ\u12c1\uf894쯸㵭騐螃᠂Лﷀ䌵팞捨랎&zip=ReKy6HPu1n&terms=',
 '/order?item=蕨㎕쇖&name=ỹ寰⻂뢐分쫽산㶓礯仸㔡髻ᘐ殏Ⴤ㽆뻄檁ꪺ杨⃧ቈ卦\uf2b1琯艷倠휒ꤧ悐쒛䞤㟦㿑\uf30f蘲\udb54汑衱䜯퍝椔穛ཁ\ue9e2&email=ձ\ud961颎ꖁ蕼쭱䏴隵赊\ud8f1襙\uf8b8ﻆ䑍껆ⲔϞꩦ囸蜷矵Ԁ&city=직ⶸ쵘蒭龹환퉋宼齘\uf834蚍ힸ术捄ꦚソ蘥킫쇘&zip=iA}B|n\\c?5&terms=嗨',
 '/order?item=⪯⢂㠃ꇔ&name=㣼鈙웲㥭\uf361аḾ뒓蚄풣ㅀ퉦箄ꠉꩭ삝ۂ斱䭭쟇ʛͰ&email=录ै걤鉴\uedfe틁皶矆鸡햗\uec51뗰蠯ꨔ棨\ue784옴諸峼&city=墄⼠볆폼\udc04旰뷴ﳥ錏仩灴縖蓚ᓘË绹毬㾤앬캠&zip=|xKfitZE$b&terms=\uabee繃劏骈ꪵ']


complex_population_2 = ['/order?item=á퓔\u0882咡&name=鈈跕殾\uf176ⶆ륡敟좽xЇꚘ돃⛮猬⟩僫讌ꂈﶘꞵ駆愜&email=Ꜧ堳湬Ά\uee58魣\uf688ᷛ硉㱞醇䋓퉍沶捁굙\ue920&city=姭찃瓦\ude0a률썃҉쉅ᕜ⥲탪⾠佖쉻\uda4dꯇ奔榩愕扤鰎꜠䚅㮾&zip=Yzg$E.+Ryn&terms=⤞玣ퟁ',
 '/order?item=ẇ\uf456鏯⪧ꞔꆋ&name=\uf64bኄ\ue28d蛆\uf546㍇憆\uf813땔벖䂘鷸ज\uead7䍥ᛆ亻&email=뒤\uee03耺띵꽆\udd46羲\uf7b1뢉너媖聁眾ꊎ䥗Ǫ⩹笽&city=᎗擟ꂀ鲗睋⊘ꄧ똅ㅰ\uda09\ue36f㤏ޫ茛讯谗驖䊝墏\ue3cd&zip=WmpE-x"Hz^&terms=鎔ﱪ뗐',
 '/order?item=ជ獿죑\udb31惖\ue9e4⾌钳ꩇ&name=勬阈\uebe6\ue6e3獶\uf0ca壇뤔髑䂂縉\ud96c㐷⢟\u09fc窔\ueaa6曱줉&email=\uf512ນ䷸낥镻飰&city=\uf700잢䞧ႁፅ嫡揃⟖肎⮸媁Ꙩ蛁溧\udc2d늙\ue899嬙괝섂蜉ꅐ痮尰&zip=zqw*JPxYU$&terms=娎ㆍ纱ﱫ',
 '/order?item=鞅\ue4fbꛧ딯ￚб&name=\uebe7⾔䫦彪Ꮓ\ue701咦琚圜ᠭ\uf3bb\uf2a0鷹ᢟ燯㔚ꆰᏌꧮ엇䎏좌횰밭̏\uf080仁ᵄ炚䩹\uf742\ue8f6靭諧ᶥ䰠&email=熶⃢\u31bb\udf77\uf2a7힊嶨ڼ쒈囖ⶇ돫\uf765&city=鍳\uf89f僻ﱊ暹㶚濛쬫\u2bf5켔ꐃ\u0a3b䕗吟ᦷ䴟ﮛ堝弽&zip=;uZ6bpZ#^d&terms=இ❆⸜斛',
 '/order?item=\udfe9&name=荿쁺丨树ੂ\uf158\uf20c䦂哾꿼焪瓹受\ude8b湇ኼോퟆᩬ䙮汔폮ҟ⇡\uecda읜貚䱚⁸ë䊴쥦擱갖╱\uf72d\ud99c&email=Ƃ\x98ꎼ맢\ueafe䭱䎟깢\uea22쏜\uee32&city=䗶莝ﾩ膕햳훣ᔅ塒ᰒ刑\u07fe毢턃&zip=Q9`wp|/r;C&terms=\ue5ee',
 '/order?item=ꓣᓜ䕩&name=\uebe8䕏\uecb9椵㦼틱䞞ﴟ\ud9ec\ue8d5宱謓ꎣ\ue435콢緕咚듌ꅉ셪ᕀ駽犨\uee17ꢴᘳ͡&email=ꇣ數&city=\udf97쏛\u242fᚓ䁰\ueb76圂醺⠏ܻ\uf66a죳냓ɪ蜵휕᧸喫뭿&zip=2OdH~^)(\\A&terms=숯奡焯\u0fe2㢯',
 '/order?item=눌\ue5e4䋙摛&name=蹷ݸ⚝䘖ⴍߊ雙ϓ貸\uf166蛮ྋ䨨鈥牆鍢杹흮媴ꁋ芻扂䅭ڀﶝ㳬郐釳闿鎜虂\uf1f8\uef03\udc4a伍&email=긋\uf10a\ue738\ue6e6䢩퐳︯⑮膘氷叒ڃᚌ嗘얒찃厇\ue1a2枵卲\u0fcd缓⪆&city=\uf25f㦺否\udb5e컉턋\u2006ᩜ圸韀\ue72aᕑ\uee19㐴ꁚ\u1cfd&zip=9I:Zb*dI,x&terms=캥↦',
 '/order?item=\u0edbどᨆ긾垠潷磄⊍&name=燒쭍ﱄ펱❰᮲\ue2ec㟏豭趛&email=⟌⤇瑅&city=붫ᓶ䟴\u173bꨜ㝚⣭&zip=scY:j&P?rn&terms=',
 '/order?item=近Ⱏ暜殌絜豙&name=슰ቇ&email=㝨\ue6ca⼩牉懟꒔墟\ue2e0ヂ╩鎄ꂌ\uf804쌋⏼螴砅Ќ&city=₥躞&zip=-ViP>2{ZKo&terms=䡴',
 '/order?item=&name=\ueee2昍襁䑜\uf063櫗庴ꉱꝒ욒免ᖼູ妾ྊх琢ᖺ垬粺뤈繀\uf41b㱊\uf57a갫&email=湂姽&city=恽&zip=HoebJmpqCh&terms=ᗁ']

complex_population_3 = ['/order?item=熹&name=\x0e굝Ᵹ贈⮽妽蓉귲츍镄莸龿\ue1e0쯌ᇡ䍎铲\x84ᒬ\ud96e༪鰊蓙\u31e6ꈉ\u0e7b\ue869婒岐뽶梦০彜\u0e68ጉጎ襼鰻䆵抇踕뚟ⷣ&email=偍\x05줚\udb58\uf6c3洫\uebd9끔\uebf0첀▂㌗\uf022㍛ᰏ䷒硓햌Ňᘱ葺ᙚ&city=㡧蚷\udbf7ꡕ쏓\ue156엠ꦂ\u0adcΓ⌻ꚦ\uf315\ue6b6싛ןڒ㯒&zip=%q87u?{S6Y&terms=뎝빅⯄㫯',
 '/order?item=&name=퐝\udd9a\u2e5e⇮交夷\udbe4᪳훏丐㚩ꖇ╿ᑦ論&email=腩ꊽ몝䷰갩✶跂ᷬ蓳訧坺ẃ킖炫\ue443睫\ue3ab꾵艰睝&city=\ue451䶑⬮敭覍⚬뙐✺ì왌薆\ue532Ⱞ\u0dff甩ཽ倖\udd9c\ue595쵄⣮谱&zip=ou2N82x]v1&terms=撵✾',
 '/order?item=\u1ccaꢣ뷭퐔쐾卸脱\udcc1㘕&name=\ue737旷춭乾⫪繌\ue423녢\udd77滺녦ꍋ뛁냡쒫喕ꞰⲟŲ枡퇫\ufddb侩\uea28眊ᨉ䋆筜噒烨腀邬&email=쑔&city=\uef76諐瀌䟪냄\uecb1鬐&zip=t/|\\=)Cc?a&terms=㔎䣪蝽鏐',
 '/order?item=&name=䎼眩穹䷨ꝩ㌥贵\udc04펦㴬䧤斂橕戻姀䀵萃좏覌諡윫ࠫ\ue0fb뮷\uea33沼賳갫榲䶗碸覒ꉋᒦིﴲ▒蓁荦\ue2dc&email=\ueb0f똩㰒\udf2bㆂ譛\uf61e\ufff0Ⱉ椲ᤤ䔳⪯ⶖ騿䔂ࠃ睒&city=\uffff&zip=8~K91!E9PS&terms=\ue651㶐摹욊\ue1b8',
 '/order?item=칯萁ꖬ\uf515䖰縛̎燙&name=\uaaff棢ǣ䮩\uf036煟ⶈ㍳Ỳ裘躀곌\ued40딙ꭆ䅃༉\u243a㽁䄑褣胖\uf5c1㕒\uf1f0픤م麗\ud92d⼊跮ᥢ恎冈频橑㪪粠睦뤸&email=\ueec7ங翷씨轲鹐䛒憇逸\uec17&city=咴ीۍ횚\u0e7d\udf2f泼\ueae9팽箂᪨䷶⋟䐳냽褳&zip=gVyw:CXO<&&terms=姴',
 '/order?item=㰱&name=䩼굉먡ꌤ낿됨ꧼ䰲踷គ霨鋻\ua63bﻆ\uf8f0ⶌ듴쇥鑲ƒ䚚쿁ン썶抍㰃䰀ූ蘑홚쨰\ud95d홫鏏\uf1d2䭍ꔐ&email=硹늕귴A皖腰좚䋧\ue3da&city=륵蝓ﰪщ&zip=3\'P)o#,HX"&terms=ͺ\ue1f4',
 '/order?item=䕇蕀\ueffc擄&name=\uf689\udc62솝\uf088&email=\ue514섻釁垰튜磙ꓖ枏&city=䱶\u0893ౙ䕙\udaa8胶ྩ\ued13돝貓뀩꼼㾼Ѳ凑皏ʉ慥穚裃䤋⦏릹&zip=!#^&FhvO1l&terms=',
 '/order?item=㻉惍㚛칤痏촊&name=ᒌ凼\uee7b★䋭&email=ꢒ髫섥㕮䄧儡ྻ詀囅ﴝ䰑ᅘ꦳\ue877큿隚⛁蜢嘓큖Ꭰ&city=\uec1c헟圻켞뻫ᷱ⢧洊䮖ᔑ⻍&zip=w_a,#*6P=T&terms=ꍘ艅岇⏴㼹',
 '/order?item=殪ᵽ뾋뇚⒕\ue1a8鶈\uf079&name=鋗넜噚&email=贤\udf70븩袢콠ꨓ甂驂ࡋ\u0c77䍍鶩䷧쉳&city=莧⤇忦䀮拜⧗ꭖ\ue246ᷫ컳\u20cb决떄&zip=koIX>r<*E$&terms=',
 '/order?item=\u0ef8ܙ㶸\u0b04❼뫻맀佈&name=\ue9aa蠤\ue910骴ဤ髨蒓爽埜력梟睑숊쯁㷹ᮝ횙✒烪쮁쥆鳌鶜ꋁ㛓閼螢\ud879곓\uf3e4䝍\uf397롞\ue2a0&email=蚾䋮⳿\uea30&city=㡾㪜\ue596\udb0e릞ߺ礪&zip=r~:yfiKW94&terms=ᯙ\uec0dꈔ汫㘏']

complex_population_4 = ['/order?item=价㎝&name=怂嚨䒩\ueb18셔럊ꑛ\ufefe婑ﶿÿƨ&email=낕ꃠﱔᳳ⥀ᬸ&city=䕍\ue577礈焧ﬀ㫾엱\uef13&zip=BjV4zD8ERU&terms=ᔨ媪',
 '/order?item=웤\uf7c3ᄼ\ue574롊㉵&name=檹\uf23f两콝遢꺒䧴ؖ\ue2ec쌀륺靵蚅諸敁２앴幑឵펾䬳孮冮勍ம퉋ᆝ搜왐䜧鈕똬䫉꼂븚촲\uecc0\ue284砶䂻겐ా챔짨맊ﲣ&email=錹ች娮㋥⣓㱾㮖\uf079譽觑枕꺏\udb12憮&city=绠抻虛힃ᙡι긻잪ᶠ빼멗ౕ␏ᬰ縪&zip=gEd73^NGQC&terms=퀌',
 '/order?item=\uf45c喭&name=ྲƧ犚\uf6e1븼ﲤ趸喠᭹晔穒坵跨楣瘇\u09d0饘\udb3a带稇\uf848䗏佈\uedb6蓠嶿ﰴ䯻ʰ鶅ὧ\ue436䚻ㆬ喉F舎嬔裲\uf826&email=톆\ua7e4朠븄决偻ᾛ峁鬝ȶ㳝&city=쯮꽉⸋苯⊅镯ť诐Ԟ篢葼&zip=!S*g%Mlh>D&terms=\ue718㌫晜랶',
 '/order?item=㉱ҵ\ue03d掹&name=\uf6c0䩕٧㊉￨鼶黢愙৸ম莿᷃닲୪⃞਼݈忢澐㬸䗴\ua6ffᴭ♤\udc15魼耏\ud810٥&email=\u2b74⭸䰓嫥큝ﭹ롃솪⏲呥\uebc1硋셝琐鸜\ue1a5Ⰰꞩ铜鬒\uef4a&city=쑹&zip=K}\\7!j#*bH&terms=㑭陕삖伣',
 "/order?item=뉫\ued79&name=鼣劭\ud8c1䳢\udf4fힰ凇Ϻ᳢㦬\u1cb3釴綯ㇺ\ue52c્ɵ࠴鳀ⓞ鹯\udb13轁욓嘮\ue24f綍ꉟ叒㉯\uf84a鲈\uef8bታ⠾滧र&email=쯭ﮨ槊Ⳙ⢏샟꾍㍔匴菟䒣\uebfb뀔놸\ue6c2馼䉓椙␠䕂㯎\uf6f1쐸鋯抅炯㿷晜&city=煌㜓&zip=Q4'@uDu/{V&terms=윇ꀿ鈾",
 '/order?item=殖鳈ꑑ폳ပ&name=ἂ﮹單懍ﯽ锼\ue316\ue263\uf80f뾽蠲ꍿ࡙ᑐ耯뙸\udb53䙜\ude08眫펀驸䁾宀㤳誀馷ⵛ釘壁ꗐ誼끓禬&email=⣈\uec5d冏꿅桝ᔸ睘뢂픾泱᧾&city=ຫ༯Ꮸ᯳ꇲוֹ쵓෧蓝疩囀吾&zip=fXtUWl_}Ou&terms=险쇤ᩒ',
 '/order?item=&name=ᾌɐ↊쪎ṣ\u0fe0\ue294饯芮鵳Ҁ西╟\uee19ꬱ膞Վ瞮\udaf0먣䠱芐\udc3dꨪ\uf016폫緾䟜㞰邵럽ꏲ邹ᠺ齍⒰形&email=㺕曣\ueaed霳\u0a63뜊妭\uf4ba悷㗬⺜䘘梇㺫엟\u0cd0嘥믽&city=轚ꄹ䡫&zip=o,{i2J1]7D&terms=',
 '/order?item=\uf502ࠁｐ\ue50c&name=乵垈읍飀튍끁ਝⵖ䝚\ue3ff䬴쒚뼠⎘︧\ue77c⯁濘\uaac4쪌&email=ა뗩槁\x83ᮩ宥彭䤅\uf25c\udf2c騌\ue334쪽듟䦇ﶻ㉭젖疥ᦶ鬟\ueca0䃖㻔ꍖ&city=텵鹟耶ʷ読䧧&zip=@WB(C2oIRS&terms=㨤㲺㹰⛛',
 '/order?item=Ն\ue736놴烁눬崵䑌&name=褒\uf5b9髰\ued41뙅\ueb14䄘칒㑳\udd3f\uf340ᰮ㐩꽀ﴣ⅗\uf5c3绹&email=㹙똾㧢㇝㪐\uebdcⅸ春ꦡ\ua7c1댇햨᳟殨죥罸夐ꆄ&city=\ud99d⪚耓\ueec1Ὂꋸ\uefb8&zip=$dr,D*H3&0&terms=镋쿋픺쾖觐',
 '/order?item=와瓞&name=\u0e7a絮湏\uee90搄\uf104ᎋ\ue1a3鱅ﮡ꼚赱蝍✲䥱✾쒿䈩㖲㨺㞊㯵瞸鯁辛沨Ō灬\uaafc狑ী\ue57f隮蘳Ꙇꃟ自猽\ueda6劬\ue2e6㹨&email=䴜괞&city=慒ဧ幧ᱝ錏ল㍈芓䡙㱛\udadc♪䎣\ue72b弊\u2bdd归笡⏫&zip=dSP_Zq_[lp&terms=蜵败赶']

complex_population_5 = ['/order?item=퐕\ue503&name=Ⴄ蜕\udd73ꝩ季\uea38黝\u0c5c賞굺\ue1f2\udfd9쥦燧Šޜ\uefbc\ue8b4꙽앚ⱥ왇\ud898ᲄ⽸驈㓞\udc5d䖋밊⮰餎Ṧ⨁촱্\uec5f&email=㶻蚑癝쬐帱ꞙ敶&city=⫇黨雏\udda4ቀ宵\u0af6䇒ᥙ뗫\ue620줺喰섚㽜䠧铮쇂\ueeb5셌罕߉ﯥ⠀&zip=Mp~:UG<e,z&terms=筺绔ᜓ抚',
 '/order?item=&name=펑ꄋ\udea9䚮⊽䳽㻘濴\u245fⶮ耓깁ꪊ\ueaac\ue2bd휑㗜㱬㹟㼵ꡀ呠쬯琿惌慦\ue068ც猥阜ᛲ\uda4c贶ꇿ\u1b7f&email=ﱮ茢롆憀☘࿚㉊鵂솪翹乹輳ꠚ넄&city=ካ\ue029٬긡륣꩖滜嘡㦈ፌ퐖唑帖\ue299↛ᖎ쓷覒৵߭믖&zip=>%&,`ckV)w&terms=蓒',
 "/order?item=\u1fdc읻솠\ue149巀欀ɹﱫ&name=碑态刷潩堒果䳎騥䲎쥻啜첁虔ऺ윸ᚶ몜䥺慮ᱫ敕兝鞖ꅙ몸\ufb45㹿뭶\ue806몚됇淙洃蠼\ue2b5琴濶\u1acc쨄&email=⟺ꤓ๛ʞ뵫槡踤먑ई㮺\u2439\ue15d⻣듞믮쫙\u1cba孢溚&city=ཪ뉋❜\ued18䃯&zip=9>'Nls_@:L&terms=",
 '/order?item=&name=丶簩⪳궯獵듥\u1a8a\ud889ટ偤\uec4a䟱꼷彊觙糌옿ජ鑑鋘䲘⭁&email=즍ꑂ蕵쥑㼼詔㆞\uf5fe␒최ᕢ猎̞&city=柵첋\ufb0d뎞꜕ꋿ&zip=co\\wJKsTMQ&terms=\udfb7쉏ᣁᙇ㎓',
 '/order?item=\ue1c8&name=巠ꔪ⦵ឿ課㜢\uef86삐麡鈚殯쩭Ⱈ碣䥷氕꤉뿦ൌ調\ue60a⥎蚭ㇺㄥ\ue604ᾄ딆軗츏ซ葸\ua97d큼䎸삽&email=炢筊Ż㘖ᒠ令䏖ᤲ쳝먣㬂&city=藶\udbc9䨝遶邺컂ꛐ멈鯉륔뺦䩈쑻⓭驅钮껠棇Ἵ긶쾓졥윢懓&zip=6Er]e.f=sV&terms=诃쪻',
 '/order?item=䢮ꯜ蚦\udae2䜼\uee53齃\uf17b&name=譨쎤\u07fb舫ƽ㹴略斴遑콖栱챘毙悺᠈餽䁘䧕睵醯⥖\ued68臅梁譧〯徿ꎇ戫湐\u0ea0౺侢ꨎ曗惆羰\uf44f᯾碟&email=ᆨ쮕Ɽ\uede9\ue5ee섗\ue689&city=㜘칍茒\ufbcd抠현\ue28cꜳ&zip=Ac6tTEB1aj&terms=掛텔᷉',
 '/order?item=&name=춏较던懾瘧Ⲣ\ue06f越✏\ue2f5짭\ue392\u0ef7榋漢ሮ፟\uf3c2ꩌٳ\ua7f6㭑퓩앑\uec26욷专ᷕ뙔фᚤ\ued12驫確槺ᦧ웍ឡ涖\uead9䜒2暹㿐龖垅肙ᝍ&email=糜\uedce\ue89d\uf6ecꚾ䦗脫譮䫢⨱Ꮑ沯䟭\uef37粎␕ⳟ䬝痙\uea19誸✝號鑱貔&city=丣ః빝흝䧵䫹뺗笹잃ꃓ㻬熜&zip=([WXZ9#B%h&terms=哞뽩',
 '/order?item=另\ue13e꧖\u173f&name=\uf435牲\ue883ᣕ䝈ꄡ\u1bf8直멞Ꝅm敐\ue489魨东Ă俹\udf38ᡀ뙄\uaafeヾ五푞糴\udec4ᾨ欘穆庴꓂ⓖ불&email=ꐹ\uf1b3玉&city=㔲⥃䡷&zip=Grp?3aO=JV&terms=넋욈眐츻',
 '/order?item=笐샨&name=ᛀ鴍螫磶瞚➬여䶟즮ꐟ樳⍲祊\u171aꦊ訄\uf658\ue8f4맅ﺑ&email=軳균쐺蹭촱⚸锭ꗽ&city=鳿籂᩶똹추䁰棻\ue10c\u0ba7＼割盕脀䠍䃱ㆶ郂ⷨ폆橅ﮊꦞꉧ&zip=Vwf4E[b?d]&terms=塥覎審鵢',
 '/order?item=\udbaf&name=㈼\uf25a\u181b常ө폸\udf46㰡鸹ᮭ⨡唛燸捣\ue313䣺&email=볨\u0cdd覣檍홨猩⟼쑦Ⰵ撀⚌木\uf636䕕驝듉\uf7df紻虰籠\ud949&city=틵푁惝︧쥲\udc74퐉ꝶ¢❺❠헾\ufffaఽ럧霙\ue92c妹瘫ꎧ&zip=-*3WV?&p2X&terms=䩌周']

In [ ]:
initial_population = [easy_population_1, easy_population_2, easy_population_3, easy_population_4, easy_population_5, \
              medium_population_1, medium_population_2, medium_population_3, medium_population_4, medium_population_5] 

harder_population = [hard_population_1, hard_population_2, hard_population_3, hard_population_4, hard_population_5, \
              complex_population_1, complex_population_2, complex_population_3, complex_population_4, complex_population_5]

## Scoring

For each URL input in the population and its corresponding target, your implementaton should generate a list of test inputs, __maximum of 10 inputs__. These inputs will be executed on the server and graded based on:

* Number of iterations of your GA algorithm (less is better)
* Number of reached target pages , i.e. error pages, confirmation page and page not found
* Number of resulting valid inputs for each target page.


__Note__
* __For each target, we expect at least _one_ valid generated input to pass the target, out of maximum 10 generated inputs__.
    
* __Your implementation will be tested with a _secret_ (possibly more complicated) population which may contain inputs other than the ones provided in the project.__ 

## Grading framework

For grading:
  * __To pass__, your implementation should produce valid inputs that reach the confirmation page, page not found and all error pages for the `easy_population` and `medium_population`.
  __Points__ will be assigned based on the number of reached pages, reaching the confirmation page, number of resulting valid inputs and the number of iterations of your GA algorithms. 
  * __To obtain extra points__, your implementation should produce valid inputs that reach the confirmation page, page not found and all error pages for all or some of the `hard_population`. 
  __Points__ will be assigned based on the number of reached pages, reaching the confirmation page, number of resulting valid inputs and the number of iterations of your GA algorithms.
  * __Bonus Points__, your implementation should produce valid inputs that reach the confirmation page, page not found and all error pages for all or some of the `complex_population`. 
  __Points__ will be assigned based on the number of reached pages, reaching the confirmation page, number of resulting valid inputs and the number of iterations of your GA algorithms.

In [ ]:
def compute_results(population):
    resulting_population = [] 
    num_generations = 0
    num_valid, num_invalid, validity_score, reachability_score = 0,0,0,0
    
    reachability_list, validity_list = [],[] 
    gen_list, pass_list = [], []
    
    dash = '-' * 115
    print(dash)
    print('{:<8s}{:<8s}{:<10s}{:>8s}{:>12s}{:>12s}{:>18s}{:>12s}{:>20s}'.format("pop_id", "reg_id", "target", "#gens", "#val", "#inval", "reachability", "validity(%)", "Pass(1)/Fail(0)"))
    print(dash)
      
    itn = 0
    num_regex = 1
    
    for reg_ind in range(0,num_regex):
        
        list_regex = [itemList, name_regex[reg_ind], email_regex[reg_ind], city_regex[reg_ind], zip_regex[reg_ind], terms_regex]
        httpd_process, httpd_url = start_httpd(ProjectHTTPRequestHandler)
        
        for test_population in population:
            itn += 1
            for target in target_list:

                try:
                    resulting_population, num_generations = genetic_algorithm(test_population, target, list_regex)        
                except Exception:
                    pass

                if len(resulting_population) <= 10:
                    reachability_score, num_valid = get_web_reachability(resulting_population, target)
                    num_invalid = len(test_population) - num_valid
                    validity_score = (num_valid/len(test_population)) * 100  

                    gen_list.append(num_generations)

                    reachability_list.append(reachability_score)
                    validity_list.append(validity_score)

                    pass_list.append(bool(validity_score > 1))

                    print('{:<8d}{:<8d}{:<10s}{:>8d}{:>10d}{:>12d}{:>16d}{:>16d}{:>16b}'\
                          .format(itn, reg_ind + 1, target, num_generations, num_valid, num_invalid, int(reachability_score), int(validity_score), \
                                validity_score > 1))
                else:
                    raise ValueError("Number of generated inputs ({0}) is greater than 10 ".format(len(resulting_population)))

    total = len(population) * len(target_list) * num_regex
    
    per_passed = (sum(pass_list)/total) * 100
    per_failed = ((total - sum(pass_list))/ total) * 100
    reachability = sum(reachability_list)
    avg_validity = (sum(validity_list)/ len(validity_list))
    avg_gens = sum(gen_list)/len(gen_list)
    
    return per_passed, per_failed, reachability, avg_validity, avg_gens

In [ ]:
per_passed, per_failed, reachability, avg_validity, avg_gens = compute_results(initial_population)

In [ ]:
print("Percent passed: {0:.2f}%".format(per_passed))
print("Percent failed: {0:.2f}%".format(per_failed))

In [ ]:
print("avg % validity: {0:.2f}%".format(avg_validity))
print("avg gen_list: {0:.2f}".format(avg_gens))

In [ ]:
print("Reachability: {0:.2f}".format(reachability))